# MISMAP SCORE with CYS/LYS positions

# cut out section on ENSP 8861 sequence comparison, move to another markdown!!!



v1 markdown = 19_09_11_Ensembl_versionIDs_IDENTITYproseq_MISMAP_SCORING.ipynb




#### isoform count file from Uniprot ID mapping: 
**previously made in markdown : 19_09_04_DEPvsINDEP_v97versionID_IDmappingdat**
- UniprotIDMAPPING_isoformCount__labeledBefore_ENSPsource_3955unique entries.csv



In [1]:
# packages
import os
import sys
import numpy as np
import pandas as pd
import csv
import argparse
import Bio
from Bio import SeqIO
from ast import literal_eval # for mismap_score func
from statistics import mean
import jellyfish
import textdistance

In [2]:
from IPython.display import display, HTML
pd.set_option('display.max_columns', None)
pd.options.display.max_seq_items = 2000
display(HTML("<style>.container {width:90% !important;}</style>"))
sys.path.append("/Users/mariapalafox/Desktop/Toolbox")
from all_funx import *

In [3]:
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/TSV_UNIPROT_xref/MULTIMAPPING_IDS")


---
---
---

# [0] make dictionary of detected positions, update includes specific C and K count columns

In [4]:
# filtering files
# for constant 3954 ukb ids in all releases but different number of ens
id3953 = pd.read_csv("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/TSV_UNIPROT_xref/MULTIMAPPING_IDS/UKBID_3953set.csv")
id3953.head(3)

,xref
0,A0AVT1
1,A0FGR8
2,A0JNW5


In [5]:
ls3953 = list(id3953.xref)

In [17]:
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CYS_LYS_2012_to2018UKBpositions/REFERENCE_2018_mappedCK/SUBSETS/")

everC = pd.read_csv("Cys2018_ever_labeled_6234.csv")
everK = pd.read_csv("Lys2018_ever_labeled_9042.csv")
#oldic = pd.read_csv("dictOfPositions_allCysLys_3963.csv")

oldic = pd.read_csv("dictOfPositions_allCysLys_3963.csv")

In [18]:
fil = [oldic, everC, everK]

In [19]:
# filtering out all none 3954 ID rows
everC = addcolumnconditionalDrop(ls3953, everC, 'ID', 'in3953')
everK = addcolumnconditionalDrop(ls3953, everK, 'ID', 'in3953')
oldic = addcolumnconditionalDrop(ls3953, oldic, 'ID', 'in3953')

  in3953  Count
0   True   6225
1  False      9
dropping false
df shape post drop:  (6225, 7)
  in3953  Count
0   True   9034
1  False      8
dropping false
df shape post drop:  (9034, 7)
  in3953  Count
0   True   3953
1  False     10
dropping false
df shape post drop:  (3953, 5)


In [52]:
# grouping to get specific ever labeled C and K counts than merging with old dic to make ultimate mismap dic
Cgroup = everC.groupby('ID')['pos'].apply(list)
Kgroup = everK.groupby('ID')['pos'].apply(list)

In [53]:
Cgroup = pd.DataFrame(Cgroup)
Cgroup.columns = ['C_pos']
Cgroup.head(3)

,C_pos
ID,
A0AVT1,"[C347, C433, C546, C625, C721, C770]"
A0FGR8,[C181]
A0MZ66,[C565]


In [54]:
Kgroup = pd.DataFrame(Kgroup)
Kgroup.columns = ['K_pos']
Kgroup.head(3)

,K_pos
ID,
A0AVT1,"[K1014, K409, K544, K86, K963]"
A0JNW5,[K645]
A1L0T0,[K538]


In [76]:
print(Cgroup.shape)
print(Kgroup.shape)
# total C K ever IDs

(2909, 2)
(2618, 2)


In [55]:
Cgroup['count_C_targets'] = Cgroup['C_pos'].apply(lambda x: len(x))
Kgroup['count_K_targets'] = Kgroup['K_pos'].apply(lambda x: len(x))

In [58]:
CKgroup = pd.concat([Cgroup, Kgroup], axis=1)
CKgroup = CKgroup[['count_C_targets', 'count_K_targets']]
CKgroup = CKgroup.fillna(0)
CKgroup['count_C_targets'] = CKgroup['count_C_targets'].astype(int)
CKgroup['count_K_targets'] = CKgroup['count_K_targets'].astype(int)

CKgroup.reset_index(inplace=True)

CKgroup.columns = ['ID', 'count_C_targets', 'count_K_targets']

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


(3953, 4)

In [69]:
# merge with old dic
dicmerge = pd.merge(oldic,CKgroup,how='inner',on=['ID'])
print(dicmerge.shape)

(3953, 7)


In [77]:
dicmerge.head(4)

,ID,pos,labeled_pos_count,pos_dict,in3953,count_C_targets,count_K_targets
0,Q09666,"['C108', 'C1833', 'C1900', 'C1967', 'C2162', '...",112,"{85: 'K', 108: 'C', 128: 'K', 243: 'K', 252: '...",True,7,105
1,P21333,"['C1018', 'C1157', 'C1165', 'C1260', 'C1353', ...",75,"{53: 'C', 58: 'K', 87: 'K', 210: 'C', 220: 'K'...",True,25,50
2,O75369,"['C1087', 'C1158', 'C1280', 'C1326', 'C1617', ...",65,"{15: 'K', 16: 'K', 26: 'C', 60: 'K', 93: 'K', ...",True,18,47
3,Q15149,"['C1098', 'C1136', 'C1405', 'C3110', 'C317', '...",58,"{191: 'K', 236: 'K', 253: 'K', 317: 'C', 338: ...",True,22,36


In [71]:
dicmerge.to_csv("dict_with_CK_counts_3953.csv", index=False)

In [72]:
dicmerge.describe()

,labeled_pos_count,count_C_targets,count_K_targets
count,3953.000000,3953.000000,3953.000000
mean,3.860106,1.574753,2.285353
std,5.027748,1.942373,3.965186
min,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,2.000000,1.000000,1.000000
75%,5.000000,2.000000,3.000000
max,112.000000,25.000000,105.000000


In [87]:
dicmerge.columns

Index(['ID', 'pos', 'labeled_pos_count', 'pos_dict', 'in3953',
       'count_C_targets', 'count_K_targets'],
      dtype='object')

In [78]:
print(dicmerge.count_C_targets.sum())
print(dicmerge.count_K_targets.sum())

6225
9034


In [82]:
def saveColumnValues(df, col, outname):
    v = df[col].value_counts().reset_index().rename(columns={'index':col, col:'Count'})
    v.to_csv(outname, index=False)
    print("saved!")
    print()
    print(v)

In [83]:
saveColumnValues(dicmerge, 'count_C_targets', 'dict_3953_C_value_counts.csv')

saved!

    count_C_targets  Count
0                 1   1522
1                 0   1044
2                 2    631
3                 3    329
4                 4    182
5                 5     96
6                 6     56
7                 7     31
8                 8     23
9                10      8
10               11      7
11                9      6
12               12      4
13               14      3
14               25      2
15               18      2
16               20      2
17               15      2
18               17      1
19               13      1
20               22      1


In [84]:
saveColumnValues(dicmerge, 'count_K_targets', 'dict_3953_K_value_counts.csv')

saved!

    count_K_targets  Count
0                 0   1335
1                 1   1088
2                 2    470
3                 3    272
4                 4    187
5                 5    130
6                 6    107
7                 7     75
8                 8     70
9                 9     39
10               11     35
11               10     32
12               13     25
13               12     16
14               15     14
15               14     11
16               16      9
17               17      7
18               19      6
19               18      5
20               24      3
21               25      2
22               28      2
23               26      2
24              105      1
25               20      1
26               22      1
27               30      1
28               36      1
29               47      1
30               50      1
31               23      1
32               27      1
33               29      1
34               42      1


In [85]:
saveColumnValues(dicmerge, 'labeled_pos_count', 'dict_3953_CK_value_counts.csv')

saved!

    labeled_pos_count  Count
0                   1   1476
1                   2    735
2                   3    441
3                   4    285
4                   5    203
5                   6    161
6                   7    123
7                   8    108
8                   9     71
9                  10     65
10                 11     55
11                 12     33
12                 13     30
13                 14     29
14                 15     22
15                 16     19
16                 17     14
17                 19     13
18                 20     11
19                 18      9
20                 21      7
21                 24      7
22                 22      6
23                 25      4
24                 29      4
25                 32      3
26                 23      3
27                 27      2
28                 36      2
29                 30      2
30                 75      1
31                 26      1
32                 42      1
33    

----
----
-----

## [1] filter UKBccds df to contain only entries from 3953 ID set


In [239]:
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/TSV_UNIPROT_xref/MULTIMAPPING_IDS")

In [89]:
ukb = pd.read_csv('/Users/mariapalafox/Box Sync/CODE_DATA/db_UKBccds_set/COUNTING_pipeline/countsfromfastaCCDS_filter_normalize_all_AA.csv')

In [91]:
refukb = addcolumnconditionalDrop(ls3953, ukb, 'ID', 'in3953')
print(refukb.shape)
print(refukb.columns)

  in3953  Count
0  False  14479
1   True   3953
dropping false
df shape post drop:  (3953, 30)
(3953, 30)
Index(['Unnamed: 0', 'ID', 'entryName', 'Length', 'proSequence', 'A', 'B', 'C',
       'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S',
       'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'in3953'],
      dtype='object')


In [92]:
# creating dictionary from ID and Sequence columns 3953 IDS
ref_dic = dict(zip(refukb.ID, refukb.proSequence))

In [93]:
ref_dic

{'Q9HAS0': 'MLPSLQESMDGDEKELESSEEGGSAEERRLEPPSSSHYCLYSYRGSRLAQQRGDSEDGSPSGTNAETPSGDDFSLSLADTNLPSEVEPELRSFIAKRLSRGAVFEGLGNVASVELKIPGYRVGCYYCLFQNEKLLPETVTIDSERNPSEYVVCFLGGSEKGLELFRLELDKYIQGLKNNMNCEARGLESHIKSYLSSWFEDVVCPIQRVVLLFQEKLTFLLHAALSYTPVEVKESDEKTKRDINRFLSVASLQGLIHEGTMTSLCMAMTEEQHKSVVIDCSSSQPQFCNAGSNRFCEDWMQAFLNGAKGGNPFLFRQVLENFKLKAIQDTNNLKRFIRQAEMNHYALFKCYMFLKNCGSGDILLKIVKVEHEEMPEAKNVIAVLEEFMKEALDQSF',
 'Q86X76': 'MLGFITRPPHRFLSLLCPGLRIPQLSVLCAQPRPRAMAISSSSCELPLVAVCQVTSTPDKQQNFKTCAELVREAARLGACLAFLPEAFDFIARDPAETLHLSEPLGGKLLEEYTQLARECGLWLSLGGFHERGQDWEQTQKIYNCHVLLNSKGAVVATYRKTHLCDVEIPGQGPMCESNSTMPGPSLESPVSTPAGKIGLAVCYDMRFPELSLALAQAGAEILTYPSAFGSITGPAHWEVLLRARAIETQCYVVAAAQCGRHHEKRASYGHSMVVDPWGTVVARCSEGPGLCLARIDLNYLRQLRRHLPVFQHRRPDLYGNLGHPLS',
 'Q9NQR4': 'MTSFRLALIQLQISSIKSDNVTRACSFIREAATQGAKIVSLPECFNSPYGAKYFPEYAEKIPGESTQKLSEVAKECSIYLIGGSIPEEDAGKLYNTCAVFGPDGTLLAKYRKIHLFDIDVPGKITFQESKTLSPGDSFSTFDTPYCRVGLGICYDMRFAELAQIYAQRGCQLLVYPGAFNLTTGPAHWELLQRSRAVDNQVYVATASPARDDKASYVAWGHSTVVNPWGEVLAKA

In [4]:
# importing fasta of ensembl sequences
v85 = pd.read_csv("v85_fasta_merge_xref_3953IDs_10183.csv")
v92 = pd.read_csv("v92_fasta_merge_xref_3953IDs_10395.csv")
v94 = pd.read_csv("v94_fasta_merge_xref_3953IDs_10612.csv")
v96 = pd.read_csv("v96_fasta_merge_xref_3953IDs_10663.csv")
v97 = pd.read_csv("v97_fasta_merge_xref_3953IDs_10564.csv")


In [6]:
v85.head(10)

,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,xref,pro_ver,tx_ver,gen_ver
0,ENSP00000313454.4,ENSP00000313454,ENST00000322244.5,ENST00000322244,ENSG00000033178.8,ENSG00000033178,1052,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,ENSG00000033178_ENST00000322244_ENSP00000313454,A0AVT1,4,5,8
1,ENSP00000399234.2,ENSP00000399234,ENST00000420827.2,ENST00000420827,ENSG00000033178.8,ENSG00000033178,389,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,ENSG00000033178_ENST00000420827_ENSP00000399234,A0AVT1,2,2,8
2,ENSP00000251527.5,ENSP00000251527,ENST00000251527.5,ENST00000251527,ENSG00000117868.11,ENSG00000117868,893,MTPPSRAEAGVRRSRVPSEGRWRGAEPPGISASTQPASAGRAARHC...,ENSG00000117868_ENST00000251527_ENSP00000251527,A0FGR8,5,5,11
3,ENSP00000279907.7,ENSP00000279907,ENST00000279907.7,ENST00000279907,ENSG00000111647.8,ENSG00000111647,1464,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...,ENSG00000111647_ENST00000279907_ENSP00000279907,A0JNW5,7,7,8
4,ENSP00000349285.3,ENSP00000349285,ENST00000356828.3,ENST00000356828,ENSG00000111647.8,ENSG00000111647,522,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...,ENSG00000111647_ENST00000356828_ENSP00000349285,A0JNW5,3,3,8
5,ENSP00000444824.2,ENSP00000444824,ENST00000545232.2,ENST00000545232,ENSG00000111647.8,ENSG00000111647,1114,MQLSFTQLTIDYYPYHKAGDSCNHWMYFSDATKTKNGWANELLHEF...,ENSG00000111647_ENST00000545232_ENSP00000444824,A0JNW5,2,2,8
6,ENSP00000260777.10,ENSP00000260777,ENST00000260777.10,ENST00000260777,ENSG00000187164.13,ENSG00000187164,456,MNSSDEEKQLQLITSLKEQAIGEYEDLRAENQKTKEKCDKIRQERD...,ENSG00000187164_ENST00000260777_ENSP00000260777,A0MZ66,10,10,13
7,ENSP00000347532.4,ENSP00000347532,ENST00000355371.4,ENST00000355371,ENSG00000187164.13,ENSG00000187164,631,MNSSDEEKQLQLITSLKEQAIGEYEDLRAENQKTKEKCDKIRQERD...,ENSG00000187164_ENST00000355371_ENSP00000347532,A0MZ66,4,4,13
8,ENSP00000376635.4,ENSP00000376635,ENST00000392901.4,ENST00000392901,ENSG00000187164.13,ENSG00000187164,498,MVIEEVNFMQNHLEIEKTCRESAEALATKLNKENKTLKRISMLYMA...,ENSG00000187164_ENST00000392901_ENSP00000376635,A0MZ66,4,4,13
9,ENSP00000376636.2,ENSP00000376636,ENST00000392903.2,ENST00000392903,ENSG00000187164.13,ENSG00000187164,649,MNSSDEEKQLQLITSLKEQAIGEYEDLRAENQKTKEKCDKIRQERD...,ENSG00000187164_ENST00000392903_ENSP00000376636,A0MZ66,2,2,13


In [7]:
v85.columns = ['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG', 'Length',
       'proSequence', 'stableID_key', 'ID', 'pro_ver', 'tx_ver', 'gen_ver']
v92.columns = ['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG', 'Length',
       'proSequence', 'stableID_key', 'ID', 'pro_ver', 'tx_ver', 'gen_ver']
v94.columns = ['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG', 'Length',
       'proSequence', 'stableID_key', 'ID', 'pro_ver', 'tx_ver', 'gen_ver']
v96.columns = ['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG', 'Length',
       'proSequence', 'stableID_key', 'ID', 'pro_ver', 'tx_ver', 'gen_ver']
v97.columns = ['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG', 'Length',
       'proSequence', 'stableID_key', 'ID', 'pro_ver', 'tx_ver', 'gen_ver']

In [8]:
df = [v85,v92,v94,v96,v97]



,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,ID,pro_ver,tx_ver,gen_ver
0,ENSP00000313454.4,ENSP00000313454,ENST00000322244.10,ENST00000322244,ENSG00000033178.13,ENSG00000033178,1052,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,ENSG00000033178_ENST00000322244_ENSP00000313454,A0AVT1,4,10,13
1,ENSP00000399234.2,ENSP00000399234,ENST00000420827.2,ENST00000420827,ENSG00000033178.13,ENSG00000033178,389,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,ENSG00000033178_ENST00000420827_ENSP00000399234,A0AVT1,2,2,13
2,ENSP00000251527.6,ENSP00000251527,ENST00000251527.10,ENST00000251527,ENSG00000117868.16,ENSG00000117868,845,MSGARGEGPEAGAGGAGGRAAPENPGGVLSVELPGLLAQLARSFAL...,ENSG00000117868_ENST00000251527_ENSP00000251527,A0FGR8,6,10,16
3,ENSP00000279907.7,ENSP00000279907,ENST00000279907.12,ENST00000279907,ENSG00000111647.13,ENSG00000111647,1464,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...,ENSG00000111647_ENST00000279907_ENSP00000279907,A0JNW5,7,12,13
4,ENSP00000349285.3,ENSP00000349285,ENST00000356828.7,ENST00000356828,ENSG00000111647.13,ENSG00000111647,522,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...,ENSG00000111647_ENST00000356828_ENSP00000349285,A0JNW5,3,7,13
5,ENSP00000260777.11,ENSP00000260777,ENST00000260777.14,ENST00000260777,ENSG00000187164.20,ENSG00000187164,571,MVIEEVNFMQNHLEIEKTCRESAEALATKLNKENKTLKRISMLYMA...,ENSG00000187164_ENST00000260777_ENSP00000260777,A0MZ66,11,14,20
6,ENSP00000347532.4,ENSP00000347532,ENST00000355371.9,ENST00000355371,ENSG00000187164.20,ENSG00000187164,631,MNSSDEEKQLQLITSLKEQAIGEYEDLRAENQKTKEKCDKIRQERD...,ENSG00000187164_ENST00000355371_ENSP00000347532,A0MZ66,4,9,20
7,ENSP00000376635.4,ENSP00000376635,ENST00000392901.10,ENST00000392901,ENSG00000187164.20,ENSG00000187164,498,MVIEEVNFMQNHLEIEKTCRESAEALATKLNKENKTLKRISMLYMA...,ENSG00000187164_ENST00000392901_ENSP00000376635,A0MZ66,4,10,20
8,ENSP00000376636.3,ENSP00000376636,ENST00000392903.3,ENST00000392903,ENSG00000187164.20,ENSG00000187164,558,MNSSDEEKQLQLITSLKEQAIGEYEDLRAENQKTKEKCDKIRQERD...,ENSG00000187164_ENST00000392903_ENSP00000376636,A0MZ66,3,3,20
9,ENSP00000480109.1,ENSP00000480109,ENST00000615301.4,ENST00000615301,ENSG00000187164.20,ENSG00000187164,456,MNSSDEEKQLQLITSLKEQAIGEYEDLRAENQKTKEKCDKIRQERD...,ENSG00000187164_ENST00000615301_ENSP00000480109,A0MZ66,1,4,20


---
---
---

In [104]:
# input df and dictionary with ID and sequence to check
# compares the sequence in df and dict value, adds column based on comparison results, compares lenght
# True is they are identical sequences for same Uniprot ID
# False is they are different, no info on HOW diff they are

# ENSP is dfalt, dfukb is 3953 ukb IDs with sequences
def identicalSequenceCheck_2(dfukb, dfalt):
    ref_dic = dict(zip(dfukb.ID, dfukb.proSequence))
    ref_len = dict(zip(dfukb.ID, dfukb.Length))
    identity = []
    lenref = []
    for index, row in dfalt.iterrows():
        altpep = row['proSequence']
        ukb_id = row['ID']
        # retrieve uniprot fasta seq using reference dictionary
        refpep = ref_dic[ukb_id]
        reflen = ref_len[ukb_id]
        str(refpep)
        if refpep == altpep:
            identity.append('True')
            lenref.append(reflen)
        if refpep != altpep: 
            identity.append('False')
            lenref.append(reflen)
    # add identity score list as new column
    dfalt.loc[:,'identical_2UKBseq'] = identity
    dfalt.loc[:,'Length_UKB'] = lenref
    dfalt.loc[:,'len_ensp_minus_ukb'] = dfalt['Length'] - dfalt['Length_UKB']
    print("new shaPe: ", dfalt.shape)
    checkColumnValues(dfalt, "identical_2UKBseq")
    return dfalt

In [105]:
v85 = identicalSequenceCheck_2(refukb, v85)
v92 = identicalSequenceCheck_2(refukb, v92)
v94 = identicalSequenceCheck_2(refukb, v94)
v96 = identicalSequenceCheck_2(refukb, v96)
v97 = identicalSequenceCheck_2(refukb, v97)

new shaPe:  (10183, 16)
  identical_2UKBseq  Count
0              True   5863
1             False   4320
new shaPe:  (10395, 16)
  identical_2UKBseq  Count
0              True   5898
1             False   4497
new shaPe:  (10612, 16)
  identical_2UKBseq  Count
0              True   6036
1             False   4576
new shaPe:  (10663, 16)
  identical_2UKBseq  Count
0              True   6067
1             False   4596
new shaPe:  (10564, 16)
  identical_2UKBseq  Count
0              True   5993
1             False   4571


In [106]:
v85.head(3)

,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,ID,pro_ver,tx_ver,gen_ver,identical_2UKBseq,Length_UKB,len_ensp_minus_ukb
0,ENSP00000313454.4,ENSP00000313454,ENST00000322244.5,ENST00000322244,ENSG00000033178.8,ENSG00000033178,1052,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,ENSG00000033178_ENST00000322244_ENSP00000313454,A0AVT1,4,5,8,True,1052,0
1,ENSP00000399234.2,ENSP00000399234,ENST00000420827.2,ENST00000420827,ENSG00000033178.8,ENSG00000033178,389,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,ENSG00000033178_ENST00000420827_ENSP00000399234,A0AVT1,2,2,8,False,1052,-663
2,ENSP00000251527.5,ENSP00000251527,ENST00000251527.5,ENST00000251527,ENSG00000117868.11,ENSG00000117868,893,MTPPSRAEAGVRRSRVPSEGRWRGAEPPGISASTQPASAGRAARHC...,ENSG00000117868_ENST00000251527_ENSP00000251527,A0FGR8,5,5,11,False,921,-28


In [107]:
v85.to_csv("ENSEMBL_SEQ_v85_3953_UKBIDs.csv")  
v92.to_csv("ENSEMBL_SEQ_v92_3953_UKBIDs.csv")  
v94.to_csv("ENSEMBL_SEQ_v94_3953_UKBIDs.csv")  
v96.to_csv("ENSEMBL_SEQ_v96_3953_UKBIDs.csv")  
v97.to_csv("ENSEMBL_SEQ_v97_3953_UKBIDs.csv")

---
---
---
---


---
---
---
---


---
---
---
---


---
---
---
---


In [108]:
v97.head(3)

,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,ID,pro_ver,tx_ver,gen_ver,identical_2UKBseq,Length_UKB,len_ensp_minus_ukb
0,ENSP00000313454.4,ENSP00000313454,ENST00000322244.10,ENST00000322244,ENSG00000033178.13,ENSG00000033178,1052,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,ENSG00000033178_ENST00000322244_ENSP00000313454,A0AVT1,4,10,13,True,1052,0
1,ENSP00000399234.2,ENSP00000399234,ENST00000420827.2,ENST00000420827,ENSG00000033178.13,ENSG00000033178,389,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,ENSG00000033178_ENST00000420827_ENSP00000399234,A0AVT1,2,2,13,False,1052,-663
2,ENSP00000251527.6,ENSP00000251527,ENST00000251527.10,ENST00000251527,ENSG00000117868.16,ENSG00000117868,845,MSGARGEGPEAGAGGAGGRAAPENPGGVLSVELPGLLAQLARSFAL...,ENSG00000117868_ENST00000251527_ENSP00000251527,A0FGR8,6,10,16,False,921,-76


## [for myidentity vs xref identity in each ensmbl cross ref file]
### Comparison of xref identity scores to my own scores made by comparing release fasta files (all have same 3979 uniprot IDs with diff # of unique ENSP multimapping) showed for the GRCh38 files the 2 identity scores are about the same
- v85 was highest identity from my alignment with 57% seq. matching canonical uniprot 
- other releases have about 56% of sequences matching

---
---
---
---


---
---
---
---


# Merge with distance scores!


In [126]:
# old annotated mismap files, i deleted all columns other than IDs and calculated dist scores to merge
dist85 = pd.read_csv("SEQ_distance_UKBvsENSP/fasta85_CanonicalUKB_identitycol_distanceScores_10272.csv")
dist92 = pd.read_csv("SEQ_distance_UKBvsENSP/fasta92_CanonicalUKB_identitycol_distanceScores_10479.csv")
dist94 = pd.read_csv("SEQ_distance_UKBvsENSP/fasta94_CanonicalUKB_identitycol_distanceScores_10699.csv")
dist96 = pd.read_csv("SEQ_distance_UKBvsENSP/fasta96_CanonicalUKB_identitycol_distanceScores_10750.csv")
dist97 = pd.read_csv("SEQ_distance_UKBvsENSP/fasta97_CanonicalUKB_identitycol_distanceScores_10650.csv")

In [127]:
dist85 = dist85[['ENSPv', 'hamming_distance', 'hamming_normalized_dist', 'levenshtein_distance','levenshtein_normalized_dist']].copy()
dist92 = dist92[['ENSPv', 'hamming_distance', 'hamming_normalized_dist', 'levenshtein_distance','levenshtein_normalized_dist']].copy()
dist94 = dist94[['ENSPv', 'hamming_distance', 'hamming_normalized_dist', 'levenshtein_distance','levenshtein_normalized_dist']].copy()
dist96 = dist96[['ENSPv', 'hamming_distance', 'hamming_normalized_dist', 'levenshtein_distance','levenshtein_normalized_dist']].copy()
dist97 = dist97[['ENSPv', 'hamming_distance', 'hamming_normalized_dist', 'levenshtein_distance','levenshtein_normalized_dist']].copy()

In [129]:
v85 = pd.merge(v85, dist85, how='inner',on=['ENSPv'])
v92 = pd.merge(v92, dist92, how='inner',on=['ENSPv'])
v94 = pd.merge(v94, dist94, how='inner',on=['ENSPv'])
v96 = pd.merge(v96, dist96, how='inner',on=['ENSPv'])
v97 = pd.merge(v97, dist97, how='inner',on=['ENSPv'])

In [130]:
v85.head(3)

,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,ID,pro_ver,tx_ver,gen_ver,identical_2UKBseq,Length_UKB,len_ensp_minus_ukb,hamming_distance,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist
0,ENSP00000313454.4,ENSP00000313454,ENST00000322244.5,ENST00000322244,ENSG00000033178.8,ENSG00000033178,1052,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,ENSG00000033178_ENST00000322244_ENSP00000313454,A0AVT1,4,5,8,True,1052,0,0.0,0.000000,0.0,0.000000
1,ENSP00000399234.2,ENSP00000399234,ENST00000420827.2,ENST00000420827,ENSG00000033178.8,ENSG00000033178,389,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,ENSG00000033178_ENST00000420827_ENSP00000399234,A0AVT1,2,2,8,False,1052,-663,684.0,0.650190,663.0,0.630228
2,ENSP00000251527.5,ENSP00000251527,ENST00000251527.5,ENST00000251527,ENSG00000117868.11,ENSG00000117868,893,MTPPSRAEAGVRRSRVPSEGRWRGAEPPGISASTQPASAGRAARHC...,ENSG00000117868_ENST00000251527_ENSP00000251527,A0FGR8,5,5,11,False,921,-28,859.0,0.932682,70.0,0.076004


# How safe is stable ID mapping- within ensembl fasta release files only - stable ID over time data 

```python 
# ENSEMBL SEQ VARIABLES
v85
v92
v94
v96
v97
```

In [131]:
v97.head(5)

,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,ID,pro_ver,tx_ver,gen_ver,identical_2UKBseq,Length_UKB,len_ensp_minus_ukb,hamming_distance,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist
0,ENSP00000313454.4,ENSP00000313454,ENST00000322244.10,ENST00000322244,ENSG00000033178.13,ENSG00000033178,1052,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,ENSG00000033178_ENST00000322244_ENSP00000313454,A0AVT1,4,10,13,True,1052,0,0.0,0.000000,0.0,0.000000
1,ENSP00000399234.2,ENSP00000399234,ENST00000420827.2,ENST00000420827,ENSG00000033178.13,ENSG00000033178,389,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...,ENSG00000033178_ENST00000420827_ENSP00000399234,A0AVT1,2,2,13,False,1052,-663,684.0,0.650190,663.0,0.630228
2,ENSP00000251527.6,ENSP00000251527,ENST00000251527.10,ENST00000251527,ENSG00000117868.16,ENSG00000117868,845,MSGARGEGPEAGAGGAGGRAAPENPGGVLSVELPGLLAQLARSFAL...,ENSG00000117868_ENST00000251527_ENSP00000251527,A0FGR8,6,10,16,False,921,-76,859.0,0.932682,77.0,0.083605
3,ENSP00000279907.7,ENSP00000279907,ENST00000279907.12,ENST00000279907,ENSG00000111647.13,ENSG00000111647,1464,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...,ENSG00000111647_ENST00000279907_ENSP00000279907,A0JNW5,7,12,13,True,1464,0,0.0,0.000000,0.0,0.000000
4,ENSP00000349285.3,ENSP00000349285,ENST00000356828.7,ENST00000356828,ENSG00000111647.13,ENSG00000111647,522,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...,ENSG00000111647_ENST00000356828_ENSP00000349285,A0JNW5,3,7,13,False,1464,-942,948.0,0.647541,942.0,0.643443


# labeling ensembl file if ENSP ID is in 8861 shared IDs

In [135]:
ID8861 = pd.read_csv("v85_fasta_merge_xref_8861IDs_3887.csv")
ID8861 = ID8861['stableID_key']

In [136]:
ls88 = list(ID8861)
set88 = set(ls88)
print(len(ls88))
print(len(set88))

8861
8861


In [138]:
def addcolumnconditional(mapList, df, dfcol, newcol): 
    mendel = []
    for g in df[dfcol]:
        if g in mapList: 
            mendel.append("True")
        else:
            mendel.append("False")
    df.loc[:,newcol] = mendel
    checkColumnValues(df, newcol)
    return df

In [139]:
v85 = addcolumnconditional(ls88, v85, 'stableID_key', 'key_8861')
v92 = addcolumnconditional(ls88, v92, 'stableID_key', 'key_8861')
v94 = addcolumnconditional(ls88, v94, 'stableID_key', 'key_8861')
v96 = addcolumnconditional(ls88, v96, 'stableID_key', 'key_8861')
v97 = addcolumnconditional(ls88, v97, 'stableID_key', 'key_8861')

  key_8861  Count
0     True   8861
1    False   1322
  key_8861  Count
0     True   8861
1    False   1534
  key_8861  Count
0     True   8861
1    False   1751
  key_8861  Count
0     True   8861
1    False   1802
  key_8861  Count
0     True   8861
1    False   1703


In [140]:
print(v85.shape)
print(v92.shape)
print(v94.shape)
print(v96.shape)
print(v97.shape)

(10183, 21)
(10395, 21)
(10612, 21)
(10663, 21)
(10564, 21)


In [141]:
dflist = [v85, v92, v94, v96, v97]
for i in dflist:
    sortbycol(i, 'ENSPv')

## how many uniprot IDs represented in shared 8861 stableID_keys from ensembl seq files == 3887 uniprot IDs

- with 3979, only controlled for same Uniprot IDs, files have different # unique ENSP multi mapping to same Uniprot IDs

In [142]:
3953 - 3887

66

In [143]:
v85.to_csv("ENSEMBL_SEQ_v85_3953_UKBIDs.csv")  
v92.to_csv("ENSEMBL_SEQ_v92_3953_UKBIDs.csv")  
v94.to_csv("ENSEMBL_SEQ_v94_3953_UKBIDs.csv")  
v96.to_csv("ENSEMBL_SEQ_v96_3953_UKBIDs.csv")  
v97.to_csv("ENSEMBL_SEQ_v97_3953_UKBIDs.csv")

---
---
---
---
---

---
---
---
---
---

---
---
---
# move this section comparing ENSP IDs to new markdown
---
---

# comparing  ENSP IDs shared between all releases, do they all have the same prosequence? does 85 have the most difference compared to other GRCh38 sequences? 
- 85 has most difference
- all ensp IDs have at least 1 different protein sequence with another release

In [ ]:
# read in files saved above
# v85.to_csv("ENSEMBL_SEQ_v85_3953_UKBIDs.csv")  
# v92.to_csv("ENSEMBL_SEQ_v92_3953_UKBIDs.csv")  
# v94.to_csv("ENSEMBL_SEQ_v94_3953_UKBIDs.csv")  
# v96.to_csv("ENSEMBL_SEQ_v96_3953_UKBIDs.csv")  
# v97.to_csv("ENSEMBL_SEQ_v97_3953_UKBIDs.csv")

In [37]:
enspv92.columns

Index(['ENSPv', 'ENSP', 'Length', 'proSequence', 'MISMAP_SCORE_ENSP',
       'UniProtSP_xref', 'Identical_UKB_seq', 'matchedUKBcanonical',
       'hammingScore', 'hamNormalizedScore', 'levenshteinScore',
       'ENSPinAllReleases'],
      dtype='object')

In [38]:
# first way of testing differences
test85_92 = enspv85.where(enspv85.proSequence==enspv92.proSequence).notna()
checkColumnValues(test85_92,'proSequence')

   proSequence  Count
0         True   8807
1        False    132


In [39]:
test85_94 = enspv85.where(enspv85.proSequence==enspv94.proSequence).notna()
checkColumnValues(test85_94,'proSequence')

   proSequence  Count
0         True   8806
1        False    133


In [40]:
test85_96 = enspv85.where(enspv85.proSequence==enspv96.proSequence).notna()
checkColumnValues(test85_96,'proSequence')

   proSequence  Count
0         True   8787
1        False    152


In [41]:
test85_97 = enspv85.where(enspv85.proSequence==enspv97.proSequence).notna()
checkColumnValues(test85_97,'proSequence')

   proSequence  Count
0         True   8776
1        False    163


In [42]:
# more robust way of testing differences
# creating dictionary from ENSP and Sequence columns in all release files
# all have the same ENSP IDs - 8939
dict_85 = dict(zip(enspv85.ENSP, enspv85.proSequence))
dict_92 = dict(zip(enspv92.ENSP, enspv92.proSequence))
dict_94 = dict(zip(enspv94.ENSP, enspv94.proSequence))
dict_96 = dict(zip(enspv96.ENSP, enspv96.proSequence))
dict_97 = dict(zip(enspv97.ENSP, enspv97.proSequence))

In [47]:
def Average(ls): 
    avg =  mean(ls) 
    finalavg = round(avg,2)
    return finalavg
def identicalSequences_inEnsembl(dfEnsp, ref_dic, newcolresult):
    res = []
    ham = []
    hamN = []
    lev = []
    serSeq = dfEnsp['proSequence'].copy()
    serID = dfEnsp['ENSP'].copy()
    for inx, val in serSeq.items():
        pep = str(val)
        p = pep.strip()
        idd = str(serID[inx])
        # check pep to dict pep sequence
        mypep = ref_dic[idd]
        str(mypep)
        #mypep = mypep.strip()
        # identical
        if mypep == p:
            res.append('True')
        # not identical to other release ENSP
        # only does score for different sequences
        if mypep != p: 
            res.append('False')
            # calc. distance scores for enesembl differing sequences
            ham.append(textdistance.hamming(mypep, p))
            # normalized hamming = # mismatched positions/ len of longer sequence
            hamN.append(textdistance.hamming.normalized_distance(mypep, p))
            # levenshtein score is edit based but not not penalized position, insertion at pos 1 is jsut 1 diff
            lev.append(textdistance.levenshtein(mypep,p))
    
    # add result col of seq match
    dfEnsp.loc[:,newcolresult] = res
    # select only False to matching seq rows
    dfFalse = dfEnsp[dfEnsp[newcolresult] == 'False']
    dfFALSE_list = dfFalse['ENSP'].tolist()
    print("ENSP IDs that have different protein sequences btw releases: ", dfFALSE_list)
    checkColumnValues(dfEnsp, newcolresult)

    print("hamming score average of different ensp sequences: ", Average(ham))
    print("hamming Normalized score average of different ensp sequences: ", Average(hamN))
    print("levenshtein score average of different ensp sequences: ", Average(lev))

    # this code was for uniprot seq comparisons
    # hamaverage = np.mean(dfFalse.hammingScore)
    # print("hamming Score average : ", hamaverage)
    # hamnormAverage = np.mean(dfFalse.hamNormalizedScore)
    # print("hamming normalized average : ", hamnormAverage)
    # levaverage = np.mean(dfFalse.levenshteinScore)
    # print("levenshtein Score average : ", levaverage)
    print()
    print("returns False for pro seq match dataframe")

    return dfFalse

In [48]:
result_92vs85 = identicalSequences_inEnsembl(enspv92, dict_85, 'proSeq_sameas85')
result_94vs85 = identicalSequences_inEnsembl(enspv94, dict_85, 'proSeq_sameas85')
result_96vs85 = identicalSequences_inEnsembl(enspv96, dict_85, 'proSeq_sameas85')
result_97vs85 = identicalSequences_inEnsembl(enspv97, dict_85, 'proSeq_sameas85')

ENSP IDs that have different protein sequences btw releases:  ['ENSP00000156825', 'ENSP00000216780', 'ENSP00000222224', 'ENSP00000225573', 'ENSP00000233242', 'ENSP00000250377', 'ENSP00000251157', 'ENSP00000260777', 'ENSP00000262878', 'ENSP00000262887', 'ENSP00000263036', 'ENSP00000264108', 'ENSP00000264202', 'ENSP00000264904', 'ENSP00000265447', 'ENSP00000266027', 'ENSP00000278616', 'ENSP00000279146', 'ENSP00000289473', 'ENSP00000291281', 'ENSP00000300648', 'ENSP00000313098', 'ENSP00000313420', 'ENSP00000319361', 'ENSP00000322938', 'ENSP00000324697', 'ENSP00000325677', 'ENSP00000328690', 'ENSP00000331023', 'ENSP00000336775', 'ENSP00000337598', 'ENSP00000338552', 'ENSP00000339247', 'ENSP00000341292', 'ENSP00000341730', 'ENSP00000342035', 'ENSP00000342351', 'ENSP00000345881', 'ENSP00000349486', 'ENSP00000350199', 'ENSP00000350563', 'ENSP00000350846', 'ENSP00000351159', 'ENSP00000351483', 'ENSP00000352575', 'ENSP00000353483', 'ENSP00000354907', 'ENSP00000357679', 'ENSP00000358832', 'ENSP0

ENSP IDs that have different protein sequences btw releases:  ['ENSP00000156825', 'ENSP00000216780', 'ENSP00000222224', 'ENSP00000225573', 'ENSP00000228284', 'ENSP00000229854', 'ENSP00000233242', 'ENSP00000250377', 'ENSP00000251157', 'ENSP00000251527', 'ENSP00000256578', 'ENSP00000260777', 'ENSP00000262878', 'ENSP00000262887', 'ENSP00000263036', 'ENSP00000263741', 'ENSP00000264108', 'ENSP00000264202', 'ENSP00000264904', 'ENSP00000265447', 'ENSP00000265537', 'ENSP00000266027', 'ENSP00000278616', 'ENSP00000279146', 'ENSP00000286091', 'ENSP00000289473', 'ENSP00000291281', 'ENSP00000300648', 'ENSP00000300933', 'ENSP00000313098', 'ENSP00000313420', 'ENSP00000319361', 'ENSP00000322938', 'ENSP00000324697', 'ENSP00000325677', 'ENSP00000328690', 'ENSP00000331023', 'ENSP00000336775', 'ENSP00000337598', 'ENSP00000338552', 'ENSP00000338806', 'ENSP00000339247', 'ENSP00000340824', 'ENSP00000341292', 'ENSP00000341730', 'ENSP00000342035', 'ENSP00000342351', 'ENSP00000345230', 'ENSP00000345881', 'ENSP0

In [49]:
result_85vs92 = identicalSequences_inEnsembl(enspv85, dict_92, 'proSeq_sameas92')
result_94vs92 = identicalSequences_inEnsembl(enspv94, dict_92, 'proSeq_sameas92')
result_96vs92 = identicalSequences_inEnsembl(enspv96, dict_92, 'proSeq_sameas92')
result_97vs92 = identicalSequences_inEnsembl(enspv97, dict_92, 'proSeq_sameas92')

ENSP IDs that have different protein sequences btw releases:  ['ENSP00000156825', 'ENSP00000216780', 'ENSP00000222224', 'ENSP00000225573', 'ENSP00000233242', 'ENSP00000250377', 'ENSP00000251157', 'ENSP00000260777', 'ENSP00000262878', 'ENSP00000262887', 'ENSP00000263036', 'ENSP00000264108', 'ENSP00000264202', 'ENSP00000264904', 'ENSP00000265447', 'ENSP00000266027', 'ENSP00000278616', 'ENSP00000279146', 'ENSP00000289473', 'ENSP00000291281', 'ENSP00000300648', 'ENSP00000313098', 'ENSP00000313420', 'ENSP00000319361', 'ENSP00000322938', 'ENSP00000324697', 'ENSP00000325677', 'ENSP00000328690', 'ENSP00000331023', 'ENSP00000336775', 'ENSP00000337598', 'ENSP00000338552', 'ENSP00000339247', 'ENSP00000341292', 'ENSP00000341730', 'ENSP00000342035', 'ENSP00000342351', 'ENSP00000345881', 'ENSP00000349486', 'ENSP00000350199', 'ENSP00000350563', 'ENSP00000350846', 'ENSP00000351159', 'ENSP00000351483', 'ENSP00000352575', 'ENSP00000353483', 'ENSP00000354907', 'ENSP00000357679', 'ENSP00000358832', 'ENSP0

In [50]:
result_85vs94 = identicalSequences_inEnsembl(enspv85, dict_94, 'proSeq_sameas94')
result_92vs94 = identicalSequences_inEnsembl(enspv92, dict_94, 'proSeq_sameas94')
result_96vs94 = identicalSequences_inEnsembl(enspv96, dict_94, 'proSeq_sameas94')
result_97vs94 = identicalSequences_inEnsembl(enspv97, dict_94, 'proSeq_sameas94')

ENSP IDs that have different protein sequences btw releases:  ['ENSP00000156825', 'ENSP00000216780', 'ENSP00000222224', 'ENSP00000225573', 'ENSP00000233242', 'ENSP00000250377', 'ENSP00000251157', 'ENSP00000260777', 'ENSP00000262878', 'ENSP00000262887', 'ENSP00000263036', 'ENSP00000264108', 'ENSP00000264202', 'ENSP00000264904', 'ENSP00000265447', 'ENSP00000266027', 'ENSP00000278616', 'ENSP00000279146', 'ENSP00000289473', 'ENSP00000291281', 'ENSP00000300648', 'ENSP00000313098', 'ENSP00000313420', 'ENSP00000319361', 'ENSP00000322938', 'ENSP00000324697', 'ENSP00000325677', 'ENSP00000328690', 'ENSP00000331023', 'ENSP00000336775', 'ENSP00000337598', 'ENSP00000338552', 'ENSP00000339247', 'ENSP00000341292', 'ENSP00000341730', 'ENSP00000342035', 'ENSP00000342351', 'ENSP00000345881', 'ENSP00000349486', 'ENSP00000350199', 'ENSP00000350563', 'ENSP00000350846', 'ENSP00000351159', 'ENSP00000351483', 'ENSP00000352575', 'ENSP00000353483', 'ENSP00000354907', 'ENSP00000357679', 'ENSP00000358832', 'ENSP0

In [51]:
result_85vs96 = identicalSequences_inEnsembl(enspv85, dict_96, 'proSeq_sameas96')
result_92vs96 = identicalSequences_inEnsembl(enspv92, dict_96, 'proSeq_sameas96')
result_94vs96 = identicalSequences_inEnsembl(enspv94, dict_96, 'proSeq_sameas96')
result_97vs96 = identicalSequences_inEnsembl(enspv97, dict_96, 'proSeq_sameas96')

ENSP IDs that have different protein sequences btw releases:  ['ENSP00000156825', 'ENSP00000216780', 'ENSP00000222224', 'ENSP00000225573', 'ENSP00000228284', 'ENSP00000229854', 'ENSP00000233242', 'ENSP00000250377', 'ENSP00000251157', 'ENSP00000251527', 'ENSP00000260777', 'ENSP00000262878', 'ENSP00000262887', 'ENSP00000263036', 'ENSP00000264108', 'ENSP00000264202', 'ENSP00000264904', 'ENSP00000265447', 'ENSP00000265537', 'ENSP00000266027', 'ENSP00000278616', 'ENSP00000279146', 'ENSP00000286091', 'ENSP00000289473', 'ENSP00000291281', 'ENSP00000300648', 'ENSP00000313098', 'ENSP00000313420', 'ENSP00000319361', 'ENSP00000322938', 'ENSP00000324697', 'ENSP00000325677', 'ENSP00000328690', 'ENSP00000331023', 'ENSP00000336775', 'ENSP00000337598', 'ENSP00000338552', 'ENSP00000338806', 'ENSP00000339247', 'ENSP00000341292', 'ENSP00000341730', 'ENSP00000342035', 'ENSP00000342351', 'ENSP00000345881', 'ENSP00000347847', 'ENSP00000349486', 'ENSP00000350199', 'ENSP00000350563', 'ENSP00000350846', 'ENSP0

In [52]:
result_85vs97 = identicalSequences_inEnsembl(enspv85, dict_97, 'proSeq_sameas97')
result_92vs97 = identicalSequences_inEnsembl(enspv92, dict_97, 'proSeq_sameas97')
result_94vs97 = identicalSequences_inEnsembl(enspv94, dict_97, 'proSeq_sameas97')
result_96vs97 = identicalSequences_inEnsembl(enspv96, dict_97, 'proSeq_sameas97')

ENSP IDs that have different protein sequences btw releases:  ['ENSP00000156825', 'ENSP00000216780', 'ENSP00000222224', 'ENSP00000225573', 'ENSP00000228284', 'ENSP00000229854', 'ENSP00000233242', 'ENSP00000250377', 'ENSP00000251157', 'ENSP00000251527', 'ENSP00000256578', 'ENSP00000260777', 'ENSP00000262878', 'ENSP00000262887', 'ENSP00000263036', 'ENSP00000263741', 'ENSP00000264108', 'ENSP00000264202', 'ENSP00000264904', 'ENSP00000265447', 'ENSP00000265537', 'ENSP00000266027', 'ENSP00000278616', 'ENSP00000279146', 'ENSP00000286091', 'ENSP00000289473', 'ENSP00000291281', 'ENSP00000300648', 'ENSP00000300933', 'ENSP00000313098', 'ENSP00000313420', 'ENSP00000319361', 'ENSP00000322938', 'ENSP00000324697', 'ENSP00000325677', 'ENSP00000328690', 'ENSP00000331023', 'ENSP00000336775', 'ENSP00000337598', 'ENSP00000338552', 'ENSP00000338806', 'ENSP00000339247', 'ENSP00000340824', 'ENSP00000341292', 'ENSP00000341730', 'ENSP00000342035', 'ENSP00000342351', 'ENSP00000345230', 'ENSP00000345881', 'ENSP0

# all output saved to AllENSPcommon8939_proSequence_differences.txt
    - dir_M/TSV../AllENSPcommon8939_proSequence_differences.txt

# are uniprot canonical sequences represetned by at least 1 ensp in each release?

## save the ENSP IDs that do not have a canonical match in each release, save ENSP IDs that do not have canonical match in ALL releases

In [54]:
dflist = [enspv85, enspv92, enspv94, enspv96, enspv97]
for i in dflist:
    sortbycol(i, 'ENSP')

In [75]:
v85ALL_ukbset = set(v85['UniProtSP_xref'].tolist())
v92ALL_ukbset = set(v92['UniProtSP_xref'].tolist())
v94ALL_ukbset = set(v94['UniProtSP_xref'].tolist())
v96ALL_ukbset = set(v96['UniProtSP_xref'].tolist())
v97ALL_ukbset = set(v97['UniProtSP_xref'].tolist())

In [76]:
print(len(v85ALL_ukbset))
print(len(v92ALL_ukbset))
print(len(v94ALL_ukbset))
print(len(v96ALL_ukbset))
print(len(v97ALL_ukbset))

3979
3979
3979
3979
3979


In [81]:
v97ALL_ukbset == v85ALL_ukbset

True

In [82]:
all3979 = list(v97ALL_ukbset)

In [56]:
v85TRUE = pd.read_csv("v85_editDistance_ensemblFasta_canonicalUKBsequence_identityScore_5925.csv")
v92TRUE = pd.read_csv("v92_editDistance_ensemblFasta_canonicalUKBsequence_identityScore_5957.csv")
v94TRUE = pd.read_csv("v94_editDistance_ensemblFasta_canonicalUKBsequence_identityScore_6097.csv")
v96TRUE = pd.read_csv("v96_editDistance_ensemblFasta_canonicalUKBsequence_identityScore_6127.csv")
v97TRUE = pd.read_csv("v97_editDistance_ensemblFasta_canonicalUKBsequence_identityScore_6052.csv")

In [58]:
v85TRUE.columns

Index(['ENSPv', 'ENSP', 'Length', 'proSequence', 'MISMAP_SCORE_ENSP',
       'UniProtSP_xref', 'Identical_UKB_seq', 'matchedUKBcanonical',
       'hammingScore', 'hamNormalizedScore', 'levenshteinScore'],
      dtype='object')

In [77]:
v85TRUE_ukbset = set(v85TRUE['UniProtSP_xref'].tolist())
v92TRUE_ukbset = set(v92TRUE['UniProtSP_xref'].tolist())
v94TRUE_ukbset = set(v94TRUE['UniProtSP_xref'].tolist())
v96TRUE_ukbset = set(v96TRUE['UniProtSP_xref'].tolist())
v97TRUE_ukbset = set(v97TRUE['UniProtSP_xref'].tolist())

In [78]:
print(len(v85TRUE_ukbset))
print(len(v92TRUE_ukbset))
print(len(v94TRUE_ukbset))
print(len(v96TRUE_ukbset))
print(len(v97TRUE_ukbset))
v85TRUE_ukbset = list(v85TRUE_ukbset)
v92TRUE_ukbset = list(v92TRUE_ukbset)
v94TRUE_ukbset = list(v94TRUE_ukbset)
v96TRUE_ukbset = list(v96TRUE_ukbset)
v97TRUE_ukbset = list(v97TRUE_ukbset)

3884
3878
3895
3890
3885


In [79]:
print(len(v85TRUE_ukbset) - 3979)
print(len(v92TRUE_ukbset) - 3979)
print(len(v94TRUE_ukbset) - 3979)
print(len(v96TRUE_ukbset) - 3979)
print(len(v97TRUE_ukbset) - 3979)

-95
-101
-84
-89
-94


In [ ]:
# which uniprot IDs do not have canonical seq match from each release
def diffList(L1, L2):
    return list(set(L1) - set(L2))


In [85]:
print(len(diffList(all3979,v85TRUE_ukbset)))

95


In [86]:
print(len(diffList(all3979,v92TRUE_ukbset)))

101


In [87]:
print(len(diffList(all3979,v94TRUE_ukbset)))

84


In [88]:
print(len(diffList(all3979,v96TRUE_ukbset)))

89


In [89]:
print(len(diffList(all3979,v97TRUE_ukbset)))

94


In [96]:
v85_nomatch = diffList(all3979,v85TRUE_ukbset)

In [97]:
v92_nomatch = diffList(all3979,v92TRUE_ukbset)

In [98]:
v94_nomatch = diffList(all3979,v94TRUE_ukbset)

In [99]:
v96_nomatch = diffList(all3979,v96TRUE_ukbset)

In [100]:
v97_nomatch = diffList(all3979,v97TRUE_ukbset)

In [101]:
# what uniprot IDs are not in any of the releases as 100% identical to canonical
shared = set(v85_nomatch) & set(v92_nomatch) & set(v94_nomatch) & set(v96_nomatch) & set(v97_nomatch)
len(shared)

49

In [105]:
shared = list(shared)
# these IDs should not be in the True files from each release at all 
shared

['P07686',
 'Q9P2N6',
 'P11532',
 'Q9BX63',
 'Q8WX93',
 'Q68E01',
 'Q96ME1',
 'O15392',
 'Q9BV68',
 'O75400',
 'Q14135',
 'Q14558',
 'Q6PKG0',
 'O43708',
 'Q03001',
 'P16278',
 'O14965',
 'Q15170',
 'Q9Y679',
 'Q9Y520',
 'Q12912',
 'P36639',
 'P30837',
 'P02765',
 'P53990',
 'A6NNF4',
 'Q9BZ29',
 'O15061',
 'P20839',
 'P17927',
 'P11182',
 'Q9UMY4',
 'Q8NBT2',
 'A0FGR8',
 'O60645',
 'Q9UNH6',
 'Q9UNH7',
 'Q8WWI1',
 'Q8NBJ7',
 'Q8NCA5',
 'Q9UJ41',
 'Q8IY17',
 'P11586',
 'Q99729',
 'Q9NRG7',
 'Q9UM54',
 'Q13459',
 'Q86TG7',
 'P18887']

In [108]:
# Checking if shared uniprot ID (not in True release set) exists in list  
# QC
def canonicalMatchNotFound(els, sharedls):
    for id in sharedls:
        ukbid = id
        if ukbid in els:
            print ("Element Exists")
    print("done checking list")

In [110]:
canonicalMatchNotFound(v85TRUE_ukbset, shared)
canonicalMatchNotFound(v92TRUE_ukbset, shared)
canonicalMatchNotFound(v94TRUE_ukbset, shared)
canonicalMatchNotFound(v96TRUE_ukbset, shared)
canonicalMatchNotFound(v97TRUE_ukbset, shared)

done checking list
done checking list
done checking list
done checking list
done checking list


In [111]:
shared


['P07686',
 'Q9P2N6',
 'P11532',
 'Q9BX63',
 'Q8WX93',
 'Q68E01',
 'Q96ME1',
 'O15392',
 'Q9BV68',
 'O75400',
 'Q14135',
 'Q14558',
 'Q6PKG0',
 'O43708',
 'Q03001',
 'P16278',
 'O14965',
 'Q15170',
 'Q9Y679',
 'Q9Y520',
 'Q12912',
 'P36639',
 'P30837',
 'P02765',
 'P53990',
 'A6NNF4',
 'Q9BZ29',
 'O15061',
 'P20839',
 'P17927',
 'P11182',
 'Q9UMY4',
 'Q8NBT2',
 'A0FGR8',
 'O60645',
 'Q9UNH6',
 'Q9UNH7',
 'Q8WWI1',
 'Q8NBJ7',
 'Q8NCA5',
 'Q9UJ41',
 'Q8IY17',
 'P11586',
 'Q99729',
 'Q9NRG7',
 'Q9UM54',
 'Q13459',
 'Q86TG7',
 'P18887']

## testing 1 Uniprot ID that is 'notin97' release fasta

In [124]:
ucan = 'MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFKYSHPHHFLKTTAALRGQVVQFKLSDIGEGIREVTVKEWYVKEGDTVSQFDSICEVQSDKASVTITSRYDGVIKKLYYNLDDIAYVGKPLVDIETEALKDSEEDVVETPAVSHDEHTHQEIKGRKTLATPAVRRLAMENNIKLSEVVGSGKDGRILKEDILNYLEKQTGAILPPSPKVEIMPPPPKPKDMTVPILVSKPPVFTGKDKTEPIKGFQKAMVKTMSAALKIPHFGYCDEIDLTELVKLREELKPIAFARGIKLSFMPFFLKAASLGLLQFPILNASVDENCQNITYKASHNIGIAMDTEQGLIVPNVKNVQICSIFDIATELNRLQKLGSVGQLSTTDLTGGTFTLSNIGSIGGTFAKPVIMPPEVAIGALGSIKAIPRFNQKGEVYKAQIMNVSWSADHRVIDGATMSRFSNLWKSYLENPAFMLLDLK'
print(len(ucan))

482


In [125]:
P11182_v97 = "MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFKYSHPHHFLKTTAALRGQVVQFKLSDIGEGIREVTVKEWYVKEGDTVSQFDSICEVQSDKASVTITSRYDGVIKKLYYNLDDIAYVGKPLVDIETEALKDSEEDVVETPAVSHDEHTHQEIKGRKTLATPAVRRLAMENNIKLSEVVGSGKDGRILKEDILNYLEKQTGAILPPSPKVEIMPPPPKPKDMTVPILVSKPPVFTGKDKTEPIKGFQKAMVKTMSAALKIPHFGYCDEIDLTELVKLREELKPIAFARGIKLSFMPFFLKAASLGLLQFPILNASVDENCQNITYKASHNIGIAMDTEQGLIVPNVKNVQICSIFDIATELNRLQKLGSVSQLSTTDLTGGTFTLSNIGSIGGTFAKPVIMPPEVAIGALGSIKAIPRFNQKGEVYKAQIMNVSWSADHRVIDGATMSRFSNLWKSYLENPAFMLLDLK"

P11182uniprot = "MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFKYSHPHHFLKTTAALRGQVVQFKLSDIGEGIREVTVKEWYVKEGDTVSQFDSICEVQSDKASVTITSRYDGVIKKLYYNLDDIAYVGKPLVDIETEALKDSEEDVVETPAVSHDEHTHQEIKGRKTLATPAVRRLAMENNIKLSEVVGSGKDGRILKEDILNYLEKQTGAILPPSPKVEIMPPPPKPKDMTVPILVSKPPVFTGKDKTEPIKGFQKAMVKTMSAALKIPHFGYCDEIDLTELVKLREELKPIAFARGIKLSFMPFFLKAASLGLLQFPILNASVDENCQNITYKASHNIGIAMDTEQGLIVPNVKNVQICSIFDIATELNRLQKLGSVGQLSTTDLTGGTFTLSNIGSIGGTFAKPVIMPPEVAIGALGSIKAIPRFNQKGEVYKAQIMNVSWSADHRVIDGATMSRFSNLWKSYLENPAFMLLDLK"

In [126]:
P11182_v97 == P11182uniprot

False

In [127]:
print(len(P11182_v97))

482


In [128]:
print(len(P11182uniprot))

482


In [138]:
import difflib
output_list = [li for li in difflib.ndiff(P11182_v97, P11182uniprot) if li[0] != ' ']
output_list

['- S', '+ G']

# checking from 49 list of uniprot IDs with no canonical match in any releases

In [116]:
test = 'A6NNF4'
v97[v97['UniProtSP_xref']== test]
# not in 97

,ENSPv,ENSP,Length,proSequence,MISMAP_SCORE_ENSP,UniProtSP_xref,Identical_UKB_seq,matchedUKBcanonical,hammingScore,hamNormalizedScore,levenshteinScore
43,ENSP00000432583.2,ENSP00000432583,77,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,True,A6NNF4,False,False,662,0.897019,661
44,ENSP00000471516.1,ENSP00000471516,616,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,True,A6NNF4,False,False,122,0.165312,122


In [117]:
v96[v96['UniProtSP_xref']== test]
# no identical match in 96

,ENSPv,ENSP,Length,proSequence,MISMAP_SCORE_ENSP,UniProtSP_xref,Identical_UKB_seq,matchedUKBcanonical,hammingScore,hamNormalizedScore,levenshteinScore
43,ENSP00000432583.2,ENSP00000432583,77,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,True,A6NNF4,False,False,662,0.897019,661
44,ENSP00000471516.1,ENSP00000471516,616,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,True,A6NNF4,False,False,122,0.165312,122


In [118]:
v94[v94['UniProtSP_xref']== test]

,ENSPv,ENSP,Length,proSequence,MISMAP_SCORE_ENSP,UniProtSP_xref,Identical_UKB_seq,matchedUKBcanonical,hammingScore,hamNormalizedScore,levenshteinScore
43,ENSP00000432583.2,ENSP00000432583,77,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,True,A6NNF4,False,False,662,0.897019,661
44,ENSP00000471516.1,ENSP00000471516,616,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,True,A6NNF4,False,False,122,0.165312,122


In [119]:
v92[v92['UniProtSP_xref']== test]

,ENSPv,ENSP,Length,proSequence,MISMAP_SCORE_ENSP,UniProtSP_xref,Identical_UKB_seq,matchedUKBcanonical,hammingScore,hamNormalizedScore,levenshteinScore
43,ENSP00000432583.2,ENSP00000432583,77,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,True,A6NNF4,False,False,662,0.897019,661
44,ENSP00000471516.1,ENSP00000471516,616,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,True,A6NNF4,False,False,122,0.165312,122


In [120]:
v85[v85['UniProtSP_xref']== test]
# v85 has ENSP with same length but differs by 1 substituion

,ENSPv,ENSP,Length,proSequence,MISMAP_SCORE_ENSP,UniProtSP_xref,Identical_UKB_seq,matchedUKBcanonical,hammingScore,hamNormalizedScore,levenshteinScore
50,ENSP00000317125.7,ENSP00000317125,738,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,True,A6NNF4,False,False,1,0.001355,1
51,ENSP00000432583.2,ENSP00000432583,77,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIA...,True,A6NNF4,False,False,662,0.897019,661


In [136]:
# for A6NNF4 same length confirming 1 substition diff
v85seq = 'MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIAVSKPDLIICLEKEKEPWNMKRDEMVDEPPGICPHFAQDIWPEQGVEDSFQKVILRRFEKCGHENLQLRKGCKSVDECKVHKEGYNGLNQCFTTTQGKASQCGKYLKVFYKFINLNRYKIRHTRKKPFKCKNCVKSFCMFSHKTQHKSIYTTEKSYKCKECGKTFNWSSTLTNHKKTHTEEKPYKCEEYGKAFNQSSNYTTHKVTHTGEKPYKCEECGKAFSQSSTLTIHKRIHTGEKPCKCEECGKAFSQPSALTIHKRMHIGEKPYKCEECGKAFVWSSTLTRHKRLHSGEKPYKCEECAKAFSQFGHLTTHRIIHTGEKPYKCEECGKAFIWPSTLTKHKRIHTGEKPYKCEECGKAFHRSSNLTKHKIIHTGEKPYKCEECGKAFIWSSNLTEHKKIHTREKPYKCEECSKAFSRSSALTTHKRMHTGEKPYKCEECGKAFSQSSTLTAHKIIHTGEKPYKCEECGKAFILSSTLSKHKRIHTGEKPYKCEECGKTFNQSSNLSTHKIIHTGEKPYKCEECGKAFNRSSNLSTHKIIHTGEKPYKCDECGKSFIWSSTLFKHKIIHTGEKPYKCEECGKAFNHSQILLHIRHKRMHTGEKPYKCEECGKSFNLSSTFIKHKVIHTGVKLYKCEECGKVFFWSSALTRHKKIHAGQQPYKWEKIGKAFNQSSHLTTDKITHIGEKSYKCE'
uniprotseq = 'MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIAVSKPDLIICLEKEKEPWNMKRDEMVDEPPGICPHFAQDIWPEQGVEDSFQKVILRRFEKCGHENLQLRKGCKSVDECKVHKEGYNGLNQCFTTTQGKASQCGKYLKVFYKFINLNRYKIRHTRKKPFKCKNCVKSFCMFSHKTQHKSIYTTEKSYKCKECGKTFNWSSTLTNHKKTHTEEKPYKCEEYGKAFNQSSNYTTHKVTHTGEKPYKCEECGKAFSQSSTLTIHKRIHTGEKPCKCEECGKAFSQPSALTIHKRMHIGEKPYKCEECGKAFVWSSTLTRHKRLHSGEKPYKCEECAKAFSQFGHLTTHRIIHTGEKPYKCEECGKAFIWPSTLTKHKRIHTGEKPYKCEECGKAFHRSSNLTKHKIIHTGEKPYKCEECGKAFIWSSNLTEHKKIHTREKPYKCEECSKAFSRSSALTTHKRMHTGEKPYKCEECGKAFSQSSTLTAHKIIHTGEKPYKCEECGKAFILSSTLSKHKRIHTGEKPYKCEECGKTFNQSSNLSTHKIIHTGEKPYKCEECGKAFNRSSNLSTHKIIHTGEKPYKCDECGKSFIWSSTLFKHKRIHTGEKPYKCEECGKAFNHSQILLHIRHKRMHTGEKPYKCEECGKSFNLSSTFIKHKVIHTGVKLYKCEECGKVFFWSSALTRHKKIHAGQQPYKWEKIGKAFNQSSHLTTDKITHIGEKSYKCE'

```bash
grep A6NNF4 MISMAP_SCORE_UniprotFastaCKabund_w_UniprotCYSLYSpositionsLabeled_3979.csv 
3664,A6NNF4,738,MGLLTFRDVAIEFSLEEWQCLDTAQKNLYRNVMLENYRNLAFLGIAVSKPDLIICLEKEKEPWNMKRDEMVDEPPGICPHFAQDIWPEQGVEDSFQKVILRRFEKCGHENLQLRKGCKSVDECKVHKEGYNGLNQCFTTTQGKASQCGKYLKVFYKFINLNRYKIRHTRKKPFKCKNCVKSFCMFSHKTQHKSIYTTEKSYKCKECGKTFNWSSTLTNHKKTHTEEKPYKCEEYGKAFNQSSNYTTHKVTHTGEKPYKCEECGKAFSQSSTLTIHKRIHTGEKPCKCEECGKAFSQPSALTIHKRMHIGEKPYKCEECGKAFVWSSTLTRHKRLHSGEKPYKCEECAKAFSQFGHLTTHRIIHTGEKPYKCEECGKAFIWPSTLTKHKRIHTGEKPYKCEECGKAFHRSSNLTKHKIIHTGEKPYKCEECGKAFIWSSNLTEHKKIHTREKPYKCEECSKAFSRSSALTTHKRMHTGEKPYKCEECGKAFSQSSTLTAHKIIHTGEKPYKCEECGKAFILSSTLSKHKRIHTGEKPYKCEECGKTFNQSSNLSTHKIIHTGEKPYKCEECGKAFNRSSNLSTHKIIHTGEKPYKCDECGKSFIWSSTLFKHKRIHTGEKPYKCEECGKAFNHSQILLHIRHKRMHTGEKPYKCEECGKSFNLSSTFIKHKVIHTGVKLYKCEECGKVFFWSSALTRHKKIHAGQQPYKWEKIGKAFNQSSHLTTDKITHIGEKSYKCE,0.065,0.1477,True,1,{55: 'C'}
```

In [137]:
import difflib
output_list = [li for li in difflib.ndiff(v85seq, uniprotseq) if li[0] != ' ']
output_list
# difference between the 2 sequences is I changed to R in uniprot seq

['- I', '+ R']

# SEE 19_09_12_QC_MISMAP_RESULTS for more quality testing on sequence comparison results

---
---
---

---
---
---

---
---
---

---
---
---

---
---
---

---
---
---

---
---
---

---
---
---

---
---
---

---
---
---

---
---
---

# BELOW CODE is MISMAP SCORING!!!

In [146]:
dicmerge.head(3)

,ID,pos,labeled_pos_count,pos_dict,in3953,count_C_targets,count_K_targets
0,Q09666,"['C108', 'C1833', 'C1900', 'C1967', 'C2162', '...",112,"{85: 'K', 108: 'C', 128: 'K', 243: 'K', 252: '...",True,7,105
1,P21333,"['C1018', 'C1157', 'C1165', 'C1260', 'C1353', ...",75,"{53: 'C', 58: 'K', 87: 'K', 210: 'C', 220: 'K'...",True,25,50
2,O75369,"['C1087', 'C1158', 'C1280', 'C1326', 'C1617', ...",65,"{15: 'K', 16: 'K', 26: 'C', 60: 'K', 93: 'K', ...",True,18,47


In [150]:
refukb = refukb[['ID', 'entryName', 'Length', 'proSequence']].copy()

In [151]:
refukb.head(3)

,ID,entryName,Length,proSequence
0,Q9HAS0,NJMU_HUMAN,396,MLPSLQESMDGDEKELESSEEGGSAEERRLEPPSSSHYCLYSYRGS...
1,Q86X76,NIT1_HUMAN,327,MLGFITRPPHRFLSLLCPGLRIPQLSVLCAQPRPRAMAISSSSCEL...
2,Q9NQR4,NIT2_HUMAN,276,MTSFRLALIQLQISSIKSDNVTRACSFIREAATQGAKIVSLPECFN...


## making ultimate uniprot reference file


In [152]:
# ultimate ukb file
reference = pd.merge(refukb, dicmerge, how='inner', on=['ID'])

In [153]:
describeMe(reference)

(3953, 10)
Index(['ID', 'entryName', 'Length', 'proSequence', 'pos', 'labeled_pos_count',
       'pos_dict', 'in3953', 'count_C_targets', 'count_K_targets'],
      dtype='object')
ID                   0
entryName            0
Length               0
proSequence          0
pos                  0
labeled_pos_count    0
pos_dict             0
in3953               0
count_C_targets      0
count_K_targets      0
dtype: int64


In [155]:
reference.head(3)

,ID,entryName,Length,proSequence,pos,labeled_pos_count,pos_dict,in3953,count_C_targets,count_K_targets
0,Q9HAS0,NJMU_HUMAN,396,MLPSLQESMDGDEKELESSEEGGSAEERRLEPPSSSHYCLYSYRGS...,['C182'],1,{182: 'C'},True,1,0
1,Q86X76,NIT1_HUMAN,327,MLGFITRPPHRFLSLLCPGLRIPQLSVLCAQPRPRAMAISSSSCEL...,"['C165', 'C203', 'K161']",3,"{161: 'K', 165: 'C', 203: 'C'}",True,2,1
2,Q9NQR4,NIT2_HUMAN,276,MTSFRLALIQLQISSIKSDNVTRACSFIREAATQGAKIVSLPECFN...,"['C146', 'C153', 'C170', 'C44', 'K123', 'K130'...",7,"{44: 'C', 52: 'K', 123: 'K', 130: 'K', 146: 'C...",True,4,3


In [156]:
reference.columns

Index(['ID', 'entryName', 'Length', 'proSequence', 'pos', 'labeled_pos_count',
       'pos_dict', 'in3953', 'count_C_targets', 'count_K_targets'],
      dtype='object')

In [154]:
reference.to_csv("ultimate_UKB_sequence_key_3953.csv", index=False)

In [159]:
# droppign the columns i dont need, like proSequence and in3979xref
UKB = reference[['ID', 'entryName', 'labeled_pos_count', 'pos_dict', 'count_C_targets', 'count_K_targets']].copy()
UKB.head(3)

,ID,entryName,labeled_pos_count,pos_dict,count_C_targets,count_K_targets
0,Q9HAS0,NJMU_HUMAN,1,{182: 'C'},1,0
1,Q86X76,NIT1_HUMAN,3,"{161: 'K', 165: 'C', 203: 'C'}",2,1
2,Q9NQR4,NIT2_HUMAN,7,"{44: 'C', 52: 'K', 123: 'K', 130: 'K', 146: 'C...",4,3


In [160]:
v85.head(2)

,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,ID,pro_ver,tx_ver,gen_ver,identical_2UKBseq,Length_UKB,len_ensp_minus_ukb,hamming_distance,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist,key_8861
0,ENSP00000000233.5,ENSP00000000233,ENST00000000233.5,ENST00000000233,ENSG00000004059.6,ENSG00000004059,180,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,ENSG00000004059_ENST00000000233_ENSP00000000233,P84085,5,5,6,True,180,0,0.0,0.0,0.0,0.0,True
1,ENSP00000001008.4,ENSP00000001008,ENST00000001008.4,ENST00000001008,ENSG00000004478.5,ENSG00000004478,459,MTAEEMKATESGAQSAPLPMEGVDISPKQDEGVLKVIKREGTGTEM...,ENSG00000004478_ENST00000001008_ENSP00000001008,Q02790,4,4,5,True,459,0,0.0,0.0,0.0,0.0,True


In [177]:
import os
import sys
import numpy as np
import pandas as pd
from ast import literal_eval

def list2col(s, colname):
    fseries = pd.DataFrame(np.array(s).reshape(-1,1))
    fseries.columns = [colname]
    return fseries

def mismap_score(enspdf, uniprot):
    # new columns to add
    count_col = []
    count_cys_col = []
    count_lys_col = []

    miss_col = []
    miss_count_c = []
    miss_count_k = []

    corrFrac_col = []
    #corrFrac_c = []
    #corrFrac_k = []

    missFrac_col = []
    #missFrac_c = []
    #missFrac_k = []

    # merge two df
    mer = pd.merge(enspdf, uniprot, how='inner', on=['ID'])

    # QC merge
    print('shape ENSEMBL df: ', enspdf.shape)
    print('shape ukb df: ', uniprot.shape)
    print('shape post merge: ', mer.shape)

    for index, row in mer.iterrows():
        ensp_len = row['Length']
        pep = row['proSequence']
        ukb_id = row['ID']
        tot_tar = row['labeled_pos_count']
        posCK = row['pos_dict']
        tot_c = int(row['count_C_targets'])
        tot_k = int(row['count_K_targets'])


        # positions found counter
        count = 0
        count_cys = 0
        count_lys = 0
        # evaluate as dictionary
        python_dict = literal_eval(posCK)

        # loop thru position dict
        for key in python_dict:
            k = int(key)
            # convert pos # to index 0-based
            i = k - 1
            # if index within ensembl protein string
            if i < int(ensp_len):
                # get AA from peptide string
                AA = pep[i]
                # get AA from pos dic
                checker = python_dict[key]
                # match, increase count
                if AA == checker:
                    count += 1
                    if AA == 'C':
                        count_cys += 1
                    if AA == 'K':
                        count_lys += 1

        # fraction correct pos
        cor_per = count/tot_tar
        #c_per = count_cys/tot_c
        #k_per = count_lys/tot_k

        # fraction missed
        miss = tot_tar - count
        missC = tot_c - count_cys
        missK = tot_k - count_lys

        miss_per = miss/tot_tar
        #miss_cper = missC/tot_c
        #miss_kper = missK/tot_k

        # add columns
        count_col.append(count)
        count_cys_col.append(count_cys)
        count_lys_col.append(count_lys)

        miss_col.append(miss)
        miss_count_c.append(missC)
        miss_count_k.append(missK)

        corrFrac_col.append(round(cor_per, 2))
        #corrFrac_c.append(c_per)
        #corrFrac_k.append(k_per)

        missFrac_col.append(round(miss_per, 2))
        #missFrac_c.append(miss_count_c)
        #missFrac_k.append(miss_count_k)

    # add columns
    count_col2 = list2col(count_col, "found_count")
    count_cys_col2 = list2col(count_cys_col, "found_count_C")
    count_lys_col2 = list2col(count_lys_col, "found_count_K")

    miss_col2 = list2col(miss_col, "missed_count")
    miss_count_c2 = list2col(miss_count_c, "missed_count_C")
    miss_count_k2 = list2col(miss_count_k, "missed_count_K")

    corrFrac_col2 = list2col(corrFrac_col, "correct_frac")
    #corrFrac_c2 = list2col(corrFrac_c, "correct_frac_C")
    #corrFrac_k2 = list2col(corrFrac_k, "correct_frac_K")

    missFrac_col2 = list2col(missFrac_col, "missed_frac")
    #missFrac_c2 = list2col(missFrac_c, "missed_frac_C")
    #missFrac_k2 = list2col(missFrac_k, "missed_frac_K")

    df = pd.concat([mer, count_col2,count_cys_col2, count_lys_col2, miss_col2,miss_count_c2, miss_count_k2, corrFrac_col2, missFrac_col2], axis=1)
    print("final df shape: ", df.shape)
    print()

    return df

In [178]:
df85 = mismap_score(v85, UKB)
df92 = mismap_score(v92, UKB)
df94 = mismap_score(v94, UKB)
df96 = mismap_score(v96, UKB)
df97 = mismap_score(v97, UKB)

shape ENSEMBL df:  (10183, 21)
shape ukb df:  (3953, 6)
shape post merge:  (10183, 26)
final df shape:  (10183, 34)

shape ENSEMBL df:  (10395, 21)
shape ukb df:  (3953, 6)
shape post merge:  (10395, 26)
final df shape:  (10395, 34)

shape ENSEMBL df:  (10612, 21)
shape ukb df:  (3953, 6)
shape post merge:  (10612, 26)
final df shape:  (10612, 34)

shape ENSEMBL df:  (10663, 21)
shape ukb df:  (3953, 6)
shape post merge:  (10663, 26)
final df shape:  (10663, 34)

shape ENSEMBL df:  (10564, 21)
shape ukb df:  (3953, 6)
shape post merge:  (10564, 26)
final df shape:  (10564, 34)



In [179]:
df85.to_csv("ENSEMBL_MISMAP_SCORED_v85_3953UKB.csv",index=False)
df92.to_csv("ENSEMBL_MISMAP_SCORED_v92_3953UKB.csv",index=False)
df94.to_csv("ENSEMBL_MISMAP_SCORED_v94_3953UKB.csv",index=False)
df96.to_csv("ENSEMBL_MISMAP_SCORED_v96_3953UKB.csv",index=False)
df97.to_csv("ENSEMBL_MISMAP_SCORED_v97_3953UKB.csv",index=False)

In [180]:
df85.head(5)

,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,ID,pro_ver,tx_ver,gen_ver,identical_2UKBseq,Length_UKB,len_ensp_minus_ukb,hamming_distance,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist,key_8861,entryName,labeled_pos_count,pos_dict,count_C_targets,count_K_targets,found_count,found_count_C,found_count_K,missed_count,missed_count_C,missed_count_K,correct_frac,missed_frac
0,ENSP00000000233.5,ENSP00000000233,ENST00000000233.5,ENST00000000233,ENSG00000004059.6,ENSG00000004059,180,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,ENSG00000004059_ENST00000000233_ENSP00000000233,P84085,5,5,6,True,180,0,0.0,0.000000,0.0,0.000000,True,ARF5_HUMAN,4,"{38: 'K', 62: 'C', 159: 'C', 179: 'K'}",2,2,4,2,2,0,0,0,1.0,0.0
1,ENSP00000001008.4,ENSP00000001008,ENST00000001008.4,ENST00000001008,ENSG00000004478.5,ENSG00000004478,459,MTAEEMKATESGAQSAPLPMEGVDISPKQDEGVLKVIKREGTGTEM...,ENSG00000004478_ENST00000001008_ENSP00000001008,Q02790,4,4,5,True,459,0,0.0,0.000000,0.0,0.000000,True,FKBP4_HUMAN,16,"{28: 'K', 103: 'C', 222: 'K', 274: 'K', 282: '...",3,13,16,3,13,0,0,0,1.0,0.0
2,ENSP00000003302.4,ENSP00000003302,ENST00000003302.4,ENST00000003302,ENSG00000048028.7,ENSG00000048028,1077,MTAELQQDDAAGAADGHGSSCQMLLNQLREITGIQDPSFLHEALKA...,ENSG00000048028_ENST00000003302_ENSP00000003302,Q96RU2,4,4,7,True,1077,0,0.0,0.000000,0.0,0.000000,True,UBP28_HUMAN,3,"{45: 'K', 171: 'C', 733: 'C'}",2,1,3,2,1,0,0,0,1.0,0.0
3,ENSP00000260188.5,ENSP00000260188,ENST00000260188.5,ENST00000260188,ENSG00000048028.7,ENSG00000048028,1045,MTAELQQDDAAGAADGHGSSCQMLLNQLREITGIQDPSFLHEALKA...,ENSG00000048028_ENST00000260188_ENSP00000260188,Q96RU2,5,5,7,False,1077,-32,292.0,0.271123,32.0,0.029712,False,UBP28_HUMAN,3,"{45: 'K', 171: 'C', 733: 'C'}",2,1,3,2,1,0,0,0,1.0,0.0
4,ENSP00000005259.4,ENSP00000005259,ENST00000005259.4,ENST00000005259,ENSG00000075790.6,ENSG00000075790,241,MTLQWAAVATFLYAEIGLILIFCLPFIPPQRWQKIFSFNVWGKIAT...,ENSG00000075790_ENST00000005259_ENSP00000005259,Q9UHQ4,4,4,6,True,241,0,0.0,0.000000,0.0,0.000000,True,BAP29_HUMAN,1,{96: 'K'},0,1,1,0,1,0,0,0,1.0,0.0


In [183]:
# saving fraction of ENSP IDs that were not identical to the uniprot sequences
df85FALSE = df85[df85['identical_2UKBseq'] == 'False']
df85FALSE.reset_index(drop=True,inplace=True)
print(df85FALSE.shape)

df92FALSE = df92[df92['identical_2UKBseq'] == "False"]
df92FALSE.reset_index(drop=True,inplace=True)
print(df92FALSE.shape)

df94FALSE = df94[df94['identical_2UKBseq'] == 'False']
df94FALSE.reset_index(drop=True,inplace=True)
print(df94FALSE.shape)

df96FALSE = df96[df96['identical_2UKBseq'] == 'False']
df96FALSE.reset_index(drop=True,inplace=True)
print(df96FALSE.shape)

df97FALSE = df97[df97['identical_2UKBseq'] == 'False']
df97FALSE.reset_index(drop=True,inplace=True)
print(df97FALSE.shape)

(4320, 34)
(4497, 34)
(4576, 34)
(4596, 34)
(4571, 34)


In [184]:
# saving fraction of ENSP IDs that were shared across all ensembl release stableID_key
df85share = df85[df85['key_8861'] == 'True']
df85share.reset_index(drop=True,inplace=True)
print(df85share.shape)

df92share = df92[df92['key_8861'] == "True"]
df92share.reset_index(drop=True,inplace=True)
print(df92share.shape)

df94share = df94[df94['key_8861'] == 'True']
df94share.reset_index(drop=True,inplace=True)
print(df94share.shape)

df96share = df96[df96['key_8861'] == 'True']
df96share.reset_index(drop=True,inplace=True)
print(df96share.shape)

df97share = df97[df97['key_8861'] == 'True']
df97share.reset_index(drop=True,inplace=True)
print(df97share.shape)

(8861, 34)
(8861, 34)
(8861, 34)
(8861, 34)
(8861, 34)


In [194]:
df85share.to_csv("ENSEMBL_MISMAP_SCORED_v85_8861shared_stableIDkeys.csv",index=False)
df92share.to_csv("ENSEMBL_MISMAP_SCORED_v92_8861shared_stableIDkeys.csv",index=False)
df94share.to_csv("ENSEMBL_MISMAP_SCORED_v94_8861shared_stableIDkeys.csv",index=False)
df96share.to_csv("ENSEMBL_MISMAP_SCORED_v96_8861shared_stableIDkeys.csv",index=False)
df97share.to_csv("ENSEMBL_MISMAP_SCORED_v97_8861shared_stableIDkeys.csv",index=False)

In [195]:
reference.shape

(3953, 10)

In [196]:
print(len(set(df85FALSE['ID'])))
print(len(set(df92FALSE['ID'])))
print(len(set(df94FALSE['ID'])))
print(len(set(df96FALSE['ID'])))
print(len(set(df97FALSE['ID'])))

1796
1796
1796
1796
1796


# controlling for different number of uniprot IDs in each ensembl release file with False identity to canonical sequence score

In [188]:
# for analysis going to make sure all FALSE release files have the same Uniprot IDs linked
f85 = set(df85FALSE['ID'].tolist())
f92 = set(df92FALSE['ID'].tolist())
f94 = set(df94FALSE['ID'].tolist())
f96 = set(df96FALSE['ID'].tolist())
f97 = set(df97FALSE['ID'].tolist())

In [189]:
sharedID = f85 & f92 & f94 & f96 & f97
print(len(sharedID))
print(len(set(sharedID)))
sharedID = list(sharedID)

1796
1796


In [190]:
def addcolumnconditional(mapList, df, dfcol, newcol): 
    mendel = []
    for g in df[dfcol]:
        if g in mapList: 
            mendel.append("True")
        else:
            mendel.append("False")
    df.loc[:,newcol] = mendel
    df.drop(df[df[newcol] == "False"].index, inplace = True)
    df.reset_index(inplace=True, drop=True)
    return df

In [191]:
sameFalse85 = addcolumnconditional(sharedID, df85FALSE, 'ID','UKBIDinFALSEReleases')
sameFalse92 = addcolumnconditional(sharedID, df92FALSE, 'ID','UKBIDinFALSEReleases')
sameFalse94 = addcolumnconditional(sharedID, df94FALSE, 'ID','UKBIDinFALSEReleases')
sameFalse96 = addcolumnconditional(sharedID, df96FALSE, 'ID','UKBIDinFALSEReleases')
sameFalse97 = addcolumnconditional(sharedID, df97FALSE, 'ID','UKBIDinFALSEReleases')

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

In [203]:
print(sameFalse85.shape)
print(sameFalse92.shape)
print(sameFalse94.shape)
print(sameFalse96.shape)
print(sameFalse97.shape)

print(df85FALSE.shape)
print(df92FALSE.shape)
print(df94FALSE.shape)
print(df96FALSE.shape)
print(df97FALSE.shape)

(4189, 35)
(4074, 35)
(4148, 35)
(4166, 35)
(4143, 35)
(4189, 35)
(4074, 35)
(4148, 35)
(4166, 35)
(4143, 35)


In [193]:
print(len(set(df85FALSE['ID'])))
print(len(set(df92FALSE['ID'])))
print(len(set(df94FALSE['ID'])))
print(len(set(df96FALSE['ID'])))
print(len(set(df97FALSE['ID'])))

1796
1796
1796
1796
1796


In [197]:
sameFalse85.to_csv("ENSEMBL_MISMAP_SCORED_v85_1796UKBIDs.csv",index=False)
sameFalse92.to_csv("ENSEMBL_MISMAP_SCORED_v92_1796UKBIDs.csv",index=False)
sameFalse94.to_csv("ENSEMBL_MISMAP_SCORED_v94_1796UKBIDs.csv",index=False)
sameFalse96.to_csv("ENSEMBL_MISMAP_SCORED_v96_1796UKBIDs.csv",index=False)
sameFalse97.to_csv("ENSEMBL_MISMAP_SCORED_v97_1796UKBIDs.csv",index=False)

In [199]:
# saving fraction of ENSP IDs that are identical to the uniprot sequences
df85TRUE = df85[df85['identical_2UKBseq'] == 'True']
df85TRUE.reset_index(drop=True,inplace=True)
print(df85TRUE.shape)

df92TRUE = df92[df92['identical_2UKBseq'] ==  'True']
df92TRUE.reset_index(drop=True,inplace=True)
print(df92TRUE.shape)

df94TRUE = df94[df94['identical_2UKBseq'] ==  'True']
df94TRUE.reset_index(drop=True,inplace=True)
print(df94TRUE.shape)

df96TRUE = df96[df96['identical_2UKBseq'] ==  'True']
df96TRUE.reset_index(drop=True,inplace=True)
print(df96TRUE.shape)

df97TRUE = df97[df97['identical_2UKBseq'] ==  'True']
df97TRUE.reset_index(drop=True,inplace=True)
print(df97TRUE.shape)

(5863, 34)
(5898, 34)
(6036, 34)
(6067, 34)
(5993, 34)


In [200]:
print(len(set(df85TRUE['ID'])))
print(len(set(df92TRUE['ID'])))
print(len(set(df94TRUE['ID'])))
print(len(set(df96TRUE['ID'])))
print(len(set(df97TRUE['ID'])))

3861
3853
3869
3865
3860


In [201]:
df85TRUE.to_csv("MISMAP_SCORED_differentNumUKBID_3861_v85_True_Identity.csv",index=False)
df92TRUE.to_csv("MISMAP_SCORED_differentNumUKBID_3853_v92_True_Identity.csv",index=False)
df94TRUE.to_csv("MISMAP_SCORED_differentNumUKBID_3869_v94_True_Identity.csv",index=False)
df96TRUE.to_csv("MISMAP_SCORED_differentNumUKBID_3865_v96_True_Identity.csv",index=False)
df97TRUE.to_csv("MISMAP_SCORED_differentNumUKBID_3860_v97_True_Identity.csv",index=False)

# 2 versions of score
a. includes all ENSP IDs linked to Uniprot ID

b. includes only ENSP that had pep seq not equal to uniprot pep seq ( these include same number of uniprot IDs 1805)

### score can be calculated 2 ways
- total correct pos/ total counted for uniprot ID
- total missed pos/ total counted for uniprot ID

In [202]:
des_df85 = df85.describe()
des_df92 = df92.describe()
des_df94 = df94.describe()
des_df96 = df96.describe()
des_df97 = df97.describe()
des_df85.to_csv("MISMAP_SCORED_described_v85.csv")
des_df92.to_csv("MISMAP_SCORED_described_v92.csv")
des_df94.to_csv("MISMAP_SCORED_described_v94.csv")
des_df96.to_csv("MISMAP_SCORED_described_v96.csv")
des_df97.to_csv("MISMAP_SCORED_described_v97.csv")

# VARIABLES: 
1. df# - for diff row count, same 3953 ukb IDs
2. sameFalse# - for diff row count, same 1796 ukb IDs
3. df#shared - for same row count, same 8861 stableID_keys

In [204]:
df85.head(2)

,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,ID,pro_ver,tx_ver,gen_ver,identical_2UKBseq,Length_UKB,len_ensp_minus_ukb,hamming_distance,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist,key_8861,entryName,labeled_pos_count,pos_dict,count_C_targets,count_K_targets,found_count,found_count_C,found_count_K,missed_count,missed_count_C,missed_count_K,correct_frac,missed_frac
0,ENSP00000000233.5,ENSP00000000233,ENST00000000233.5,ENST00000000233,ENSG00000004059.6,ENSG00000004059,180,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,ENSG00000004059_ENST00000000233_ENSP00000000233,P84085,5,5,6,True,180,0,0.0,0.0,0.0,0.0,True,ARF5_HUMAN,4,"{38: 'K', 62: 'C', 159: 'C', 179: 'K'}",2,2,4,2,2,0,0,0,1.0,0.0
1,ENSP00000001008.4,ENSP00000001008,ENST00000001008.4,ENST00000001008,ENSG00000004478.5,ENSG00000004478,459,MTAEEMKATESGAQSAPLPMEGVDISPKQDEGVLKVIKREGTGTEM...,ENSG00000004478_ENST00000001008_ENSP00000001008,Q02790,4,4,5,True,459,0,0.0,0.0,0.0,0.0,True,FKBP4_HUMAN,16,"{28: 'K', 103: 'C', 222: 'K', 274: 'K', 282: '...",3,13,16,3,13,0,0,0,1.0,0.0


In [205]:
sameFalse85.head(2)

,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,ID,pro_ver,tx_ver,gen_ver,identical_2UKBseq,Length_UKB,len_ensp_minus_ukb,hamming_distance,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist,key_8861,entryName,labeled_pos_count,pos_dict,count_C_targets,count_K_targets,found_count,found_count_C,found_count_K,missed_count,missed_count_C,missed_count_K,correct_frac,missed_frac,UKBIDinFALSEReleases
0,ENSP00000260188.5,ENSP00000260188,ENST00000260188.5,ENST00000260188,ENSG00000048028.7,ENSG00000048028,1045,MTAELQQDDAAGAADGHGSSCQMLLNQLREITGIQDPSFLHEALKA...,ENSG00000048028_ENST00000260188_ENSP00000260188,Q96RU2,5,5,7,False,1077,-32,292.0,0.271123,32.0,0.029712,False,UBP28_HUMAN,3,"{45: 'K', 171: 'C', 733: 'C'}",2,1,3,2,1,0,0,0,1.0,0.0,True
1,ENSP00000368414.2,ENSP00000368414,ENST00000379119.2,ENST00000379119,ENSG00000075790.6,ENSG00000075790,348,MTLQWAAVATFLYAEIGLILIFCLPFIPPQRWQKIFSFNVWGKIAT...,ENSG00000075790_ENST00000379119_ENSP00000368414,Q9UHQ4,2,2,6,False,241,107,118.0,0.339080,110.0,0.316092,True,BAP29_HUMAN,1,{96: 'K'},0,1,1,0,1,0,0,0,1.0,0.0,True


In [208]:
df85share.head(2)

,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,ID,pro_ver,tx_ver,gen_ver,identical_2UKBseq,Length_UKB,len_ensp_minus_ukb,hamming_distance,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist,key_8861,entryName,labeled_pos_count,pos_dict,count_C_targets,count_K_targets,found_count,found_count_C,found_count_K,missed_count,missed_count_C,missed_count_K,correct_frac,missed_frac
0,ENSP00000000233.5,ENSP00000000233,ENST00000000233.5,ENST00000000233,ENSG00000004059.6,ENSG00000004059,180,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,ENSG00000004059_ENST00000000233_ENSP00000000233,P84085,5,5,6,True,180,0,0.0,0.0,0.0,0.0,True,ARF5_HUMAN,4,"{38: 'K', 62: 'C', 159: 'C', 179: 'K'}",2,2,4,2,2,0,0,0,1.0,0.0
1,ENSP00000001008.4,ENSP00000001008,ENST00000001008.4,ENST00000001008,ENSG00000004478.5,ENSG00000004478,459,MTAEEMKATESGAQSAPLPMEGVDISPKQDEGVLKVIKREGTGTEM...,ENSG00000004478_ENST00000001008_ENSP00000001008,Q02790,4,4,5,True,459,0,0.0,0.0,0.0,0.0,True,FKBP4_HUMAN,16,"{28: 'K', 103: 'C', 222: 'K', 274: 'K', 282: '...",3,13,16,3,13,0,0,0,1.0,0.0


# all 3953 ukb ID dataframes

In [225]:
grdf85 = df85.groupby(['ID'])['labeled_pos_count', 'count_C_targets', 'count_K_targets', 'found_count', 'found_count_C', 'found_count_K', 'missed_count', 'missed_count_C', 'missed_count_K'].sum().reset_index()
grdf92 = df92.groupby(['ID'])['labeled_pos_count', 'count_C_targets', 'count_K_targets', 'found_count', 'found_count_C', 'found_count_K', 'missed_count', 'missed_count_C', 'missed_count_K'].sum().reset_index()
grdf94 = df94.groupby(['ID'])['labeled_pos_count', 'count_C_targets', 'count_K_targets', 'found_count', 'found_count_C', 'found_count_K', 'missed_count', 'missed_count_C', 'missed_count_K'].sum().reset_index()
grdf96 = df96.groupby(['ID'])['labeled_pos_count', 'count_C_targets', 'count_K_targets', 'found_count', 'found_count_C', 'found_count_K', 'missed_count', 'missed_count_C', 'missed_count_K'].sum().reset_index()
grdf97 = df97.groupby(['ID'])['labeled_pos_count', 'count_C_targets', 'count_K_targets', 'found_count', 'found_count_C', 'found_count_K', 'missed_count', 'missed_count_C', 'missed_count_K'].sum().reset_index()

In [226]:
grdf85.head(3)

,ID,labeled_pos_count,count_C_targets,count_K_targets,found_count,found_count_C,found_count_K,missed_count,missed_count_C,missed_count_K
0,A0AVT1,22,12,10,13,7,6,9,5,4
1,A0FGR8,1,1,0,0,0,0,1,1,0
2,A0JNW5,3,0,3,1,0,1,2,0,2


# all 8861 stableID_key dataframes

In [227]:
grdf85share = df85share.groupby(['ID'])['labeled_pos_count', 'count_C_targets', 'count_K_targets', 'found_count', 'found_count_C', 'found_count_K', 'missed_count', 'missed_count_C', 'missed_count_K'].sum().reset_index()
grdf92share = df92share.groupby(['ID'])['labeled_pos_count', 'count_C_targets', 'count_K_targets', 'found_count', 'found_count_C', 'found_count_K', 'missed_count', 'missed_count_C', 'missed_count_K'].sum().reset_index()
grdf94share = df94share.groupby(['ID'])['labeled_pos_count', 'count_C_targets', 'count_K_targets', 'found_count', 'found_count_C', 'found_count_K', 'missed_count', 'missed_count_C', 'missed_count_K'].sum().reset_index()
grdf96share = df96share.groupby(['ID'])['labeled_pos_count', 'count_C_targets', 'count_K_targets', 'found_count', 'found_count_C', 'found_count_K', 'missed_count', 'missed_count_C', 'missed_count_K'].sum().reset_index()
grdf97share = df97share.groupby(['ID'])['labeled_pos_count', 'count_C_targets', 'count_K_targets', 'found_count', 'found_count_C', 'found_count_K', 'missed_count', 'missed_count_C', 'missed_count_K'].sum().reset_index()

# all 1796 FALSE identity UKBID shared 

In [228]:
grdf85false = sameFalse85.groupby(['ID'])['labeled_pos_count', 'count_C_targets', 'count_K_targets', 'found_count', 'found_count_C', 'found_count_K', 'missed_count', 'missed_count_C', 'missed_count_K'].sum().reset_index()
grdf92false = sameFalse92.groupby(['ID'])['labeled_pos_count', 'count_C_targets', 'count_K_targets', 'found_count', 'found_count_C', 'found_count_K', 'missed_count', 'missed_count_C', 'missed_count_K'].sum().reset_index()
grdf94false = sameFalse94.groupby(['ID'])['labeled_pos_count', 'count_C_targets', 'count_K_targets', 'found_count', 'found_count_C', 'found_count_K', 'missed_count', 'missed_count_C', 'missed_count_K'].sum().reset_index()
grdf96false = sameFalse96.groupby(['ID'])['labeled_pos_count', 'count_C_targets', 'count_K_targets', 'found_count', 'found_count_C', 'found_count_K', 'missed_count', 'missed_count_C', 'missed_count_K'].sum().reset_index()
grdf97false = sameFalse97.groupby(['ID'])['labeled_pos_count', 'count_C_targets', 'count_K_targets', 'found_count', 'found_count_C', 'found_count_K', 'missed_count', 'missed_count_C', 'missed_count_K'].sum().reset_index()

In [229]:
grdf85false.head(3)

,ID,labeled_pos_count,count_C_targets,count_K_targets,found_count,found_count_C,found_count_K,missed_count,missed_count_C,missed_count_K
0,A0AVT1,11,6,5,2,1,1,9,5,4
1,A0FGR8,1,1,0,0,0,0,1,1,0
2,A0JNW5,2,0,2,0,0,0,2,0,2


# comparing all release files FALSE for matching canonical sequence, there are 1796 uniprot IDs shared between all release files

In [230]:
dflist = [grdf85,grdf92,grdf94,grdf96,grdf97]
for i in dflist:
    print(describeMe(i))

(3953, 10)
Index(['ID', 'labeled_pos_count', 'count_C_targets', 'count_K_targets',
       'found_count', 'found_count_C', 'found_count_K', 'missed_count',
       'missed_count_C', 'missed_count_K'],
      dtype='object')
ID                   0
labeled_pos_count    0
count_C_targets      0
count_K_targets      0
found_count          0
found_count_C        0
found_count_K        0
missed_count         0
missed_count_C       0
missed_count_K       0
dtype: int64
None
(3953, 10)
Index(['ID', 'labeled_pos_count', 'count_C_targets', 'count_K_targets',
       'found_count', 'found_count_C', 'found_count_K', 'missed_count',
       'missed_count_C', 'missed_count_K'],
      dtype='object')
ID                   0
labeled_pos_count    0
count_C_targets      0
count_K_targets      0
found_count          0
found_count_C        0
found_count_K        0
missed_count         0
missed_count_C       0
missed_count_K       0
dtype: int64
None
(3953, 10)
Index(['ID', 'labeled_pos_count', 'count_C_targets'

In [231]:
dflist = [grdf85false,grdf92false,grdf94false,grdf96false,grdf97false]
for i in dflist:
    print(describeMe(i))

(1796, 10)
Index(['ID', 'labeled_pos_count', 'count_C_targets', 'count_K_targets',
       'found_count', 'found_count_C', 'found_count_K', 'missed_count',
       'missed_count_C', 'missed_count_K'],
      dtype='object')
ID                   0
labeled_pos_count    0
count_C_targets      0
count_K_targets      0
found_count          0
found_count_C        0
found_count_K        0
missed_count         0
missed_count_C       0
missed_count_K       0
dtype: int64
None
(1796, 10)
Index(['ID', 'labeled_pos_count', 'count_C_targets', 'count_K_targets',
       'found_count', 'found_count_C', 'found_count_K', 'missed_count',
       'missed_count_C', 'missed_count_K'],
      dtype='object')
ID                   0
labeled_pos_count    0
count_C_targets      0
count_K_targets      0
found_count          0
found_count_C        0
found_count_K        0
missed_count         0
missed_count_C       0
missed_count_K       0
dtype: int64
None
(1796, 10)
Index(['ID', 'labeled_pos_count', 'count_C_targets'

In [232]:
dflist = [grdf85share,grdf92share,grdf94share,grdf96share,grdf97share]
for i in dflist:
    print(describeMe(i))

(3887, 10)
Index(['ID', 'labeled_pos_count', 'count_C_targets', 'count_K_targets',
       'found_count', 'found_count_C', 'found_count_K', 'missed_count',
       'missed_count_C', 'missed_count_K'],
      dtype='object')
ID                   0
labeled_pos_count    0
count_C_targets      0
count_K_targets      0
found_count          0
found_count_C        0
found_count_K        0
missed_count         0
missed_count_C       0
missed_count_K       0
dtype: int64
None
(3887, 10)
Index(['ID', 'labeled_pos_count', 'count_C_targets', 'count_K_targets',
       'found_count', 'found_count_C', 'found_count_K', 'missed_count',
       'missed_count_C', 'missed_count_K'],
      dtype='object')
ID                   0
labeled_pos_count    0
count_C_targets      0
count_K_targets      0
found_count          0
found_count_C        0
found_count_K        0
missed_count         0
missed_count_C       0
missed_count_K       0
dtype: int64
None
(3887, 10)
Index(['ID', 'labeled_pos_count', 'count_C_targets'

In [233]:
def divideCol(df):
    df['frac_found'] = df['found_count']/df['labeled_pos_count']
    df['frac_missed'] = df['missed_count']/df['labeled_pos_count']
    return df

In [223]:
# 3979 uniprot IDs
frac_85 = divideCol(grdf85)
frac_92 = divideCol(grdf92)
frac_94 = divideCol(grdf94)
frac_96 = divideCol(grdf96)
frac_97 = divideCol(grdf97)

fracfal_85 = divideCol(grdf85false)
fracfal_92 = divideCol(grdf92false)
fracfal_94 = divideCol(grdf94false)
fracfal_96 = divideCol(grdf96false)
fracfal_97 = divideCol(grdf97false)


fracshare_85 = divideCol(grdf85share)
fracshare_92 = divideCol(grdf92share)
fracshare_94 = divideCol(grdf94share)
fracshare_96 = divideCol(grdf96share)
fracshare_97 = divideCol(grdf97share)

In [234]:
frac_85.head(5)

,ID,labeled_pos_count,found_count,found_count_C,found_count_K,missed_count,missed_count_C,missed_count_K,frac_found,frac_missed
0,A0AVT1,22,13,7,6,9,5,4,0.590909,0.409091
1,A0FGR8,1,0,0,0,1,1,0,0.000000,1.000000
2,A0JNW5,3,1,0,1,2,0,2,0.333333,0.666667
3,A0MZ66,4,2,2,0,2,2,0,0.500000,0.500000
4,A1KXE4,2,2,2,0,0,0,0,1.000000,0.000000


In [235]:
print(fracfal_85.shape)
print(fracfal_92.shape)
print(fracfal_94.shape)
print(fracfal_96.shape)
print(fracfal_97.shape)

(1796, 10)
(1796, 10)
(1796, 10)
(1796, 10)
(1796, 10)


In [236]:
# saving grouped scores, all rows including identical matches to uniprot canonical sequence
frac_85.to_csv("groupedMISMAP_score_85_3953.csv",index=False)
frac_92.to_csv("groupedMISMAP_score_92_3953.csv",index=False)
frac_94.to_csv("groupedMISMAP_score_94_3953.csv",index=False)
frac_96.to_csv("groupedMISMAP_score_96_3953.csv",index=False)
frac_97.to_csv("groupedMISMAP_score_97_3953.csv",index=False)

# saving grouped scores, only rows that were not identical to uniprot canonical sequence
fracfal_85.to_csv("groupedMISMAP_score_85_FALSEidentity_1796.csv",index=False)
fracfal_92.to_csv("groupedMISMAP_score_92_FALSEidentity_1796.csv",index=False)
fracfal_94.to_csv("groupedMISMAP_score_94_FALSEidentity_1796.csv",index=False)
fracfal_96.to_csv("groupedMISMAP_score_96_FALSEidentity_1796.csv",index=False)
fracfal_97.to_csv("groupedMISMAP_score_97_FALSEidentity_1796.csv",index=False)

fracshare_85.to_csv("groupedMISMAP_score_85_shared_stableID_key_3887.csv",index=False)
fracshare_92.to_csv("groupedMISMAP_score_92_shared_stableID_key_3887.csv",index=False)
fracshare_94.to_csv("groupedMISMAP_score_94_shared_stableID_key_3887.csv",index=False)
fracshare_96.to_csv("groupedMISMAP_score_96_shared_stableID_key_3887.csv",index=False)
fracshare_97.to_csv("groupedMISMAP_score_97_shared_stableID_key_3887.csv",index=False)

In [240]:
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/TSV_UNIPROT_xref/MULTIMAPPING_IDS")

In [241]:
# converted files into Rfigure friendly format 
# from pyconvert_Rfriendly.py

import pandas as pd 
import numpy as np 
import csv

def combineDFS(optio):
    # both true and false identity scores
    if optio == "all":
        df85=pd.read_csv("groupedMISMAP_score_85_3953.csv")
        df92=pd.read_csv("groupedMISMAP_score_92_3953.csv")
        df94=pd.read_csv("groupedMISMAP_score_94_3953.csv")
        df96=pd.read_csv("groupedMISMAP_score_96_3953.csv")
        df97=pd.read_csv("groupedMISMAP_score_97_3953.csv")
        df85['release'] = 85
        df92['release'] = 92
        df94['release'] = 94
        df96['release'] = 96
        df97['release'] = 97
        # combining dfs on axis 
        df_merge = pd.concat([df85, df92, df94, df96, df97])
        print(df_merge.head(10))
        print(df_merge.tail(10))
        print(df_merge.shape)
        print("all identity scores shape should match x5 = : ", 3953*5)
        df_merge.to_csv("Rmerge_MISMAP_AllIdentityOnly_3953.csv",index=False)

    if optio == "false":
        #only false
        df85 = pd.read_csv("groupedMISMAP_score_85_FALSEidentity_1796.csv")
        df92 = pd.read_csv("groupedMISMAP_score_92_FALSEidentity_1796.csv")
        df94 = pd.read_csv("groupedMISMAP_score_94_FALSEidentity_1796.csv")
        df96 = pd.read_csv("groupedMISMAP_score_96_FALSEidentity_1796.csv")
        df97 = pd.read_csv("groupedMISMAP_score_97_FALSEidentity_1796.csv")
        df85['release'] = 85
        df92['release'] = 92
        df94['release'] = 94
        df96['release'] = 96
        df97['release'] = 97
        # combining dfs on axis 
        df_merge = pd.concat([df85, df92, df94, df96, df97])
        print(df_merge.head(10))
        print(df_merge.tail(10))
        print(df_merge.shape)
        print("false identity scores shape should match x5 = : ", 1796*5)
        df_merge.to_csv("Rmerge_MISMAP_FalseIdentityOnly_1796.csv",index=False)

    if optio == "shared":
        df85 = pd.read_csv("groupedMISMAP_score_85_shared_stableID_key_3887.csv")
        df92 = pd.read_csv("groupedMISMAP_score_92_shared_stableID_key_3887.csv")
        df94 = pd.read_csv("groupedMISMAP_score_94_shared_stableID_key_3887.csv")
        df96 = pd.read_csv("groupedMISMAP_score_96_shared_stableID_key_3887.csv")
        df97 = pd.read_csv("groupedMISMAP_score_97_shared_stableID_key_3887.csv")
        df85['release'] = 85
        df92['release'] = 92
        df94['release'] = 94
        df96['release'] = 96
        df97['release'] = 97
        # combining dfs on axis 
        df_merge = pd.concat([df85, df92, df94, df96, df97])
        print(df_merge.head(10))
        print(df_merge.tail(10))
        print(df_merge.shape)
        print("shared 8861 stableIDs scores shape should match x5 = : ", 3887*5)
        df_merge.to_csv("Rmerge_MISMAP_sharedStableID_keyOnly_3887.csv",index=False)

def main():
    combineDFS("all")
    combineDFS("false")
    combineDFS("shared")
    print("done")
main()

       ID  labeled_pos_count  found_count  found_count_C  found_count_K  \
0  A0AVT1                 22           13              7              6   
1  A0FGR8                  1            0              0              0   
2  A0JNW5                  3            1              0              1   
3  A0MZ66                  4            2              2              0   
4  A1KXE4                  2            2              2              0   
5  A1L0T0                  1            1              0              1   
6  A2A288                  9            7              7              0   
7  A2RUC4                  1            1              0              1   
8  A3KN83                  4            2              2              0   
9  A4D1E9                  3            1              0              1   

   missed_count  missed_count_C  missed_count_K  frac_found  frac_missed  \
0             9               5               4    0.590909     0.409091   
1             1       

# STOP HERE!

In [242]:
# merging all the releases together in order to substract out lines with slope
merall = "Rmerge_MISMAP_AllIdentityOnly_3953.csv"
merfalse = "Rmerge_MISMAP_FalseIdentityOnly_1796.csv"
mershared = "Rmerge_MISMAP_sharedStableID_keyOnly_3887.csv"
mall = pd.read_csv(merall)
mfal = pd.read_csv(merfalse)
mshar = pd.read_csv(mershared)
describeMe (mall)
describeMe(mfal)
describeMe(mshar)

(19765, 11)
Index(['ID', 'labeled_pos_count', 'found_count', 'found_count_C',
       'found_count_K', 'missed_count', 'missed_count_C', 'missed_count_K',
       'frac_found', 'frac_missed', 'release'],
      dtype='object')
ID                   0
labeled_pos_count    0
found_count          0
found_count_C        0
found_count_K        0
missed_count         0
missed_count_C       0
missed_count_K       0
frac_found           0
frac_missed          0
release              0
dtype: int64
(8980, 11)
Index(['ID', 'labeled_pos_count', 'found_count', 'found_count_C',
       'found_count_K', 'missed_count', 'missed_count_C', 'missed_count_K',
       'frac_found', 'frac_missed', 'release'],
      dtype='object')
ID                   0
labeled_pos_count    0
found_count          0
found_count_C        0
found_count_K        0
missed_count         0
missed_count_C       0
missed_count_K       0
frac_found           0
frac_missed          0
release              0
dtype: int64
(19435, 11)
Index(['I

---


In [251]:
def group_scores(df):
    gdf = df.groupby('ID')['frac_missed'].apply(list)
    gdf = pd.DataFrame(gdf, index=None)
    gdf.reset_index(inplace=True)
    print("group df shaep: ", gdf.shape)
    print(gdf.head(1))
    return gdf

gall = group_scores(mall)
gfal = group_scores(mfal)
gshar = group_scores(mshar)

group df shaep:  (3953, 2)
       ID                                        frac_missed
0  A0AVT1  [0.4090909090909091, 0.4090909090909091, 0.409...
group df shaep:  (1796, 2)
       ID                                        frac_missed
0  A0AVT1  [0.8181818181818182, 0.8181818181818182, 0.818...
group df shaep:  (3887, 2)
       ID                                        frac_missed
0  A0AVT1  [0.4090909090909091, 0.4090909090909091, 0.409...


---
# dynamic slope score based on changing 'frac_missed' column
- dynamic 1 = all fraction scores in 5 releases match
- dynamic 2 = 2 unique fraction scores in 5 releases
- dynamic 3 = 3 unique ..
- dynamic 4 = 4 unique ..
- dynamic 5 does not exist because no uniprot ID have 5 unique scores for all 5 releases

In [252]:
# perviously used to check slope 0 and dynmic slope - 4 values out of 5 are unique
# expanding this out to include dynamic levels - 1 2 3 
def mismaplines_dynamic(df):
    diffline = []
    for index, row in df.iterrows():
        ukbid = row['ID']
        ls = row['frac_missed']
        #python_ls = literal_eval(ls) 
        python_ls = ls
        lenLS = len(set(python_ls))
        if lenLS == 5:
            diffline.append("5")
        if lenLS == 4:
            diffline.append("4")
        if lenLS == 3:
            diffline.append("3")
        if lenLS == 2:
            diffline.append("2")
        if lenLS == 1:
            diffline.append("1")
    df.loc[:,'dynamic_slope_scores'] = diffline
    print(df.shape)

In [253]:
mismaplines_dynamic(gall)
checkColumnValues(gall, 'dynamic_slope_scores')
describeMe(gall)
print()
mismaplines_dynamic(gfal)
checkColumnValues(gfal, 'dynamic_slope_scores')
describeMe(gfal)
print()
mismaplines_dynamic(gshar)
checkColumnValues(gshar, 'dynamic_slope_scores')
describeMe(gshar)

(3953, 3)
  dynamic_slope_scores  Count
0                    1   3051
1                    2    848
2                    3     43
3                    4     11
(3953, 3)
Index(['ID', 'frac_missed', 'dynamic_slope_scores'], dtype='object')
ID                      0
frac_missed             0
dynamic_slope_scores    0
dtype: int64

(1796, 3)
  dynamic_slope_scores  Count
0                    1   1515
1                    2    268
2                    3     12
3                    4      1
(1796, 3)
Index(['ID', 'frac_missed', 'dynamic_slope_scores'], dtype='object')
ID                      0
frac_missed             0
dynamic_slope_scores    0
dtype: int64

(3887, 3)
  dynamic_slope_scores  Count
0                    1   3824
1                    2     63
(3887, 3)
Index(['ID', 'frac_missed', 'dynamic_slope_scores'], dtype='object')
ID                      0
frac_missed             0
dynamic_slope_scores    0
dtype: int64


## dynamic slope results comments:
* in group with only False identity, only 1 ID has score of 4
* controlling of all same stable IDs, 98% all have same score

In [254]:
# saving files that will be used to label all release merge file, x5 versions of these files
gall.to_csv("R_dynamic_slope_scores_3973.csv",index=False) 

gfal.to_csv("R_dynamic_slope_scores_1796.csv",index=False) 

gshar.to_csv("R_dynamic_slope_scores_3887.csv",index=False) 

# using the dynamic slope scores column added to groupedby UKBID datasets to label each UKB ID in file not grouped by UKBID (length is either 3979 x 5 or 1805 x 5...each release has score for each UKBID)

In [255]:
# creating a dictionary of uniprotID:dynamic_slope_scores in order to map these values 
all_ref_slope = dict(zip(gall.ID, gall.dynamic_slope_scores))
false_ref_slope = dict(zip(gfal.ID, gfal.dynamic_slope_scores))
share_ref_slope = dict(zip(gshar.ID, gshar.dynamic_slope_scores))

In [259]:
# duplicatinm uniprotID column inorder to convert these IDs into there dynamicslopescores with dictionary
mall['dynamic_slope_scores'] = mall['ID']
mfal['dynamic_slope_scores'] = mfal['ID']
mshar['dynamic_slope_scores'] = mshar['ID']

mall.dynamic_slope_scores = mall.dynamic_slope_scores.map(all_ref_slope)
mfal.dynamic_slope_scores = mfal.dynamic_slope_scores.map(false_ref_slope)
mshar.dynamic_slope_scores = mshar.dynamic_slope_scores.map(share_ref_slope)

In [260]:
mall.head(3)

,ID,labeled_pos_count,found_count,found_count_C,found_count_K,missed_count,missed_count_C,missed_count_K,frac_found,frac_missed,release,dynamic_slope_scores
0,A0AVT1,22,13,7,6,9,5,4,0.590909,0.409091,85,1
1,A0FGR8,1,0,0,0,1,1,0,0.000000,1.000000,85,1
2,A0JNW5,3,1,0,1,2,0,2,0.333333,0.666667,85,2


In [261]:
mfal.head(3)

,ID,labeled_pos_count,found_count,found_count_C,found_count_K,missed_count,missed_count_C,missed_count_K,frac_found,frac_missed,release,dynamic_slope_scores
0,A0AVT1,11,2,1,1,9,5,4,0.181818,0.818182,85,1
1,A0FGR8,1,0,0,0,1,1,0,0.000000,1.000000,85,1
2,A0JNW5,2,0,0,0,2,0,2,0.000000,1.000000,85,1


In [265]:
gshar.head(3)

,ID,frac_missed,dynamic_slope_scores
0,A0AVT1,"[0.4090909090909091, 0.4090909090909091, 0.409...",1
1,A0FGR8,"[1.0, 1.0, 1.0, 1.0, 1.0]",1
2,A0JNW5,"[0.5, 0.5, 0.5, 0.5, 0.5]",1


---
---
---
---
---



# calculating absolute slope values for each uniprot ID

In [263]:
UKB.head(4)

,ID,entryName,labeled_pos_count,pos_dict,count_C_targets,count_K_targets
0,Q9HAS0,NJMU_HUMAN,1,{182: 'C'},1,0
1,Q86X76,NIT1_HUMAN,3,"{161: 'K', 165: 'C', 203: 'C'}",2,1
2,Q9NQR4,NIT2_HUMAN,7,"{44: 'C', 52: 'K', 123: 'K', 130: 'K', 146: 'C...",4,3
3,Q9Y314,NOSIP_HUMAN,3,"{8: 'C', 78: 'K', 185: 'C'}",2,1


In [264]:
# create dict of uniprotID and number of targets assocaited then add this to false_labeled and all_labeled
ref_targets = dict(zip(UKB.ID, UKB.labeled_pos_count))

len(ref_targets)

3953

In [266]:
# adding total targets column to each file
mall['tot_target_per_UKBID'] = mall['ID']
mfal['tot_target_per_UKBID'] = mfal['ID']
mshar['tot_target_per_UKBID'] = mshar['ID']

mall.tot_target_per_UKBID = mall.tot_target_per_UKBID.map(ref_targets)
mfal.tot_target_per_UKBID = mfal.tot_target_per_UKBID.map(ref_targets)
mshar.tot_target_per_UKBID = mshar.tot_target_per_UKBID.map(ref_targets)

In [267]:
# adding number of ENSP IDs linked to each UKB ID for each release
mall['ENSPID_count'] = mall['labeled_pos_count'] / mall['tot_target_per_UKBID']
mfal['ENSPID_count'] = mfal['labeled_pos_count'] / mfal['tot_target_per_UKBID']
mshar['ENSPID_count'] = mshar['labeled_pos_count'] / mshar['tot_target_per_UKBID']



mall['ENSPID_count'] = mall['ENSPID_count'].astype(int)
mfal['ENSPID_count'] = mfal['ENSPID_count'].astype(int)
mshar['ENSPID_count'] = mshar['ENSPID_count'].astype(int)

In [268]:
mfal[mfal['ID'] == 'P08397']

,ID,labeled_pos_count,found_count,found_count_C,found_count_K,missed_count,missed_count_C,missed_count_K,frac_found,frac_missed,release,dynamic_slope_scores,tot_target_per_UKBID,ENSPID_count
280,P08397,50,24,0,24,26,10,16,0.480000,0.520000,85,3,5,10
2076,P08397,50,24,0,24,26,10,16,0.480000,0.520000,92,3,5,10
3872,P08397,50,20,0,20,30,10,20,0.400000,0.600000,94,3,5,10
5668,P08397,60,28,0,28,32,12,20,0.466667,0.533333,96,3,5,12
7464,P08397,50,24,0,24,26,10,16,0.480000,0.520000,97,3,5,10


In [269]:
mall.corr()

,labeled_pos_count,found_count,found_count_C,found_count_K,missed_count,missed_count_C,missed_count_K,frac_found,frac_missed,release,tot_target_per_UKBID,ENSPID_count
labeled_pos_count,1.000000,0.865232,0.678123,0.825031,0.708659,0.612809,0.706948,-0.141085,0.141085,5.172421e-03,6.313039e-01,0.474314
found_count,0.865232,1.000000,0.787007,0.951919,0.259411,0.189756,0.280336,0.043336,-0.043336,3.154739e-03,5.693005e-01,0.435230
found_count_C,0.678123,0.787007,1.000000,0.560166,0.198725,0.236569,0.157892,0.021607,-0.021607,2.396049e-03,4.063911e-01,0.389943
found_count_K,0.825031,0.951919,0.560166,1.000000,0.249636,0.137317,0.298009,0.047459,-0.047459,3.046134e-03,5.626107e-01,0.390760
missed_count,0.708659,0.259411,0.198725,0.249636,1.000000,0.913390,0.967256,-0.332749,0.332749,5.523886e-03,4.149053e-01,0.301174
missed_count_C,0.612809,0.189756,0.236569,0.137317,0.913390,1.000000,0.780161,-0.378153,0.378153,5.203402e-03,3.002184e-01,0.326982
missed_count_K,0.706948,0.280336,0.157892,0.298009,0.967256,0.780161,1.000000,-0.275577,0.275577,5.244540e-03,4.504185e-01,0.258959
frac_found,-0.141085,0.043336,0.021607,0.047459,-0.332749,-0.378153,-0.275577,1.000000,-1.000000,-2.822579e-02,-8.195086e-03,-0.346295
frac_missed,0.141085,-0.043336,-0.021607,-0.047459,0.332749,0.378153,0.275577,-1.000000,1.000000,2.822579e-02,8.195086e-03,0.346295
release,0.005172,0.003155,0.002396,0.003046,0.005524,0.005203,0.005245,-0.028226,0.028226,1.000000e+00,-7.114500e-16,0.017100


In [270]:
mfal.corr()

,labeled_pos_count,found_count,found_count_C,found_count_K,missed_count,missed_count_C,missed_count_K,frac_found,frac_missed,release,tot_target_per_UKBID,ENSPID_count
labeled_pos_count,1.000000,0.592150,0.391312,0.579858,0.921853,0.824046,0.902202,-0.032614,0.032614,-6.423020e-03,6.473975e-01,0.389076
found_count,0.592150,1.000000,0.761878,0.917024,0.233585,0.171712,0.251354,0.271840,-0.271840,-5.175029e-03,4.905125e-01,0.270279
found_count_C,0.391312,0.761878,1.000000,0.440328,0.105766,0.131626,0.080767,0.275088,-0.275088,-5.181371e-03,2.904518e-01,0.271681
found_count_K,0.579858,0.917024,0.440328,1.000000,0.258658,0.156969,0.298682,0.207425,-0.207425,-3.982939e-03,5.010780e-01,0.207359
missed_count,0.921853,0.233585,0.105766,0.258658,1.000000,0.911739,0.967743,-0.170087,0.170087,-5.261430e-03,5.452716e-01,0.339488
missed_count_C,0.824046,0.171712,0.131626,0.156969,0.911739,1.000000,0.778840,-0.197067,0.197067,-5.864452e-03,3.978357e-01,0.375380
missed_count_K,0.902202,0.251354,0.080767,0.298682,0.967743,0.778840,1.000000,-0.138845,0.138845,-4.437039e-03,5.885928e-01,0.288146
frac_found,-0.032614,0.271840,0.275088,0.207425,-0.170087,-0.197067,-0.138845,1.000000,-1.000000,-3.781109e-03,-4.455938e-02,-0.039168
frac_missed,0.032614,-0.271840,-0.275088,-0.207425,0.170087,0.197067,0.138845,-1.000000,1.000000,3.781109e-03,4.455938e-02,0.039168
release,-0.006423,-0.005175,-0.005181,-0.003983,-0.005261,-0.005864,-0.004437,-0.003781,0.003781,1.000000e+00,-5.984869e-16,-0.002811


In [271]:
mshar.corr()

,labeled_pos_count,found_count,found_count_C,found_count_K,missed_count,missed_count_C,missed_count_K,frac_found,frac_missed,release,tot_target_per_UKBID,ENSPID_count
labeled_pos_count,1.000000e+00,0.875335,0.687050,0.836305,0.683253,0.586246,0.684202,-0.137676,0.137676,8.877469e-16,6.545334e-01,4.613479e-01
found_count,8.753355e-01,1.000000,0.786968,0.954410,0.245021,0.173740,0.268205,0.045675,-0.045675,-5.411063e-04,5.971528e-01,4.168030e-01
found_count_C,6.870496e-01,0.786968,1.000000,0.566917,0.189192,0.225505,0.149909,0.030348,-0.030348,-4.121032e-04,4.312797e-01,3.942940e-01
found_count_K,8.363050e-01,0.954410,0.566917,1.000000,0.235607,0.122869,0.285566,0.046300,-0.046300,-5.230797e-04,5.886323e-01,3.657332e-01
missed_count,6.832528e-01,0.245021,0.189192,0.235607,1.000000,0.913132,0.966892,-0.345036,0.345036,8.171518e-04,4.106412e-01,2.956322e-01
missed_count_C,5.862459e-01,0.173740,0.225505,0.122869,0.913132,1.000000,0.778869,-0.392446,0.392446,5.542412e-04,2.906274e-01,3.267552e-01
missed_count_K,6.842018e-01,0.268205,0.149909,0.285566,0.966892,0.778869,1.000000,-0.285173,0.285173,9.102382e-04,4.498417e-01,2.502864e-01
frac_found,-1.376764e-01,0.045675,0.030348,0.046300,-0.345036,-0.392446,-0.285173,1.000000,-1.000000,-4.774156e-04,1.089790e-04,-3.558403e-01
frac_missed,1.376764e-01,-0.045675,-0.030348,-0.046300,0.345036,0.392446,0.285173,-1.000000,1.000000,4.774156e-04,-1.089790e-04,3.558403e-01
release,8.877469e-16,-0.000541,-0.000412,-0.000523,0.000817,0.000554,0.000910,-0.000477,0.000477,1.000000e+00,-1.637710e-15,7.193588e-17


In [272]:
# saving files that will be loaded into R to make laser figures
mall.to_csv("Rmerge_MISMAP_SCORED_3953_X5.csv", index=False)
mfal.to_csv("Rmerge_MISMAP_SCORED_1796_X5.csv", index=False)
mshar.to_csv("Rmerge_MISMAP_SCORED_3887_X5.csv", index=False)

# PANEL 3 Release Level summaries of missed_fraction for i) all positions ii) only Cysteines ii) only Lysine

In [4]:
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/TSV_UNIPROT_xref/MULTIMAPPING_IDS/MISMAP2.0/")

In [5]:
v97s = pd.read_csv("ENSEMBL_MISMAP_SCORED_v97_8861shared_stableIDkeys.csv")
v97a = pd.read_csv("ENSEMBL_MISMAP_SCORED_v97_3953UKB.csv")
v97f = pd.read_csv("ENSEMBL_MISMAP_SCORED_v97_1796UKBIDs.csv")
v96s = pd.read_csv("ENSEMBL_MISMAP_SCORED_v96_8861shared_stableIDkeys.csv")
v96a = pd.read_csv("ENSEMBL_MISMAP_SCORED_v96_3953UKB.csv")
v96f = pd.read_csv("ENSEMBL_MISMAP_SCORED_v96_1796UKBIDs.csv")
v94s = pd.read_csv("ENSEMBL_MISMAP_SCORED_v94_8861shared_stableIDkeys.csv")
v94a = pd.read_csv("ENSEMBL_MISMAP_SCORED_v94_3953UKB.csv")
v94f = pd.read_csv("ENSEMBL_MISMAP_SCORED_v94_1796UKBIDs.csv")
v92s = pd.read_csv("ENSEMBL_MISMAP_SCORED_v92_8861shared_stableIDkeys.csv")
v92a = pd.read_csv("ENSEMBL_MISMAP_SCORED_v92_3953UKB.csv")
v92f = pd.read_csv("ENSEMBL_MISMAP_SCORED_v92_1796UKBIDs.csv")
v85s = pd.read_csv("ENSEMBL_MISMAP_SCORED_v85_8861shared_stableIDkeys.csv")
v85f = pd.read_csv("ENSEMBL_MISMAP_SCORED_v85_1796UKBIDs.csv")
v85a = pd.read_csv("ENSEMBL_MISMAP_SCORED_v85_3953UKB.csv")

In [6]:
v85a.head(3)

,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,ID,pro_ver,tx_ver,gen_ver,identical_2UKBseq,Length_UKB,len_ensp_minus_ukb,hamming_distance,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist,key_8861,entryName,labeled_pos_count,pos_dict,count_C_targets,count_K_targets,found_count,found_count_C,found_count_K,missed_count,missed_count_C,missed_count_K,correct_frac,missed_frac
0,ENSP00000000233.5,ENSP00000000233,ENST00000000233.5,ENST00000000233,ENSG00000004059.6,ENSG00000004059,180,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,ENSG00000004059_ENST00000000233_ENSP00000000233,P84085,5,5,6,True,180,0,0.0,0.0,0.0,0.0,True,ARF5_HUMAN,4,"{38: 'K', 62: 'C', 159: 'C', 179: 'K'}",2,2,4,2,2,0,0,0,1.0,0.0
1,ENSP00000001008.4,ENSP00000001008,ENST00000001008.4,ENST00000001008,ENSG00000004478.5,ENSG00000004478,459,MTAEEMKATESGAQSAPLPMEGVDISPKQDEGVLKVIKREGTGTEM...,ENSG00000004478_ENST00000001008_ENSP00000001008,Q02790,4,4,5,True,459,0,0.0,0.0,0.0,0.0,True,FKBP4_HUMAN,16,"{28: 'K', 103: 'C', 222: 'K', 274: 'K', 282: '...",3,13,16,3,13,0,0,0,1.0,0.0
2,ENSP00000003302.4,ENSP00000003302,ENST00000003302.4,ENST00000003302,ENSG00000048028.7,ENSG00000048028,1077,MTAELQQDDAAGAADGHGSSCQMLLNQLREITGIQDPSFLHEALKA...,ENSG00000048028_ENST00000003302_ENSP00000003302,Q96RU2,4,4,7,True,1077,0,0.0,0.0,0.0,0.0,True,UBP28_HUMAN,3,"{45: 'K', 171: 'C', 733: 'C'}",2,1,3,2,1,0,0,0,1.0,0.0


In [7]:
print(v97s.labeled_pos_count.sum())
print(v97a.labeled_pos_count.sum())
print(v97f.labeled_pos_count.sum())
print("done 97 s a f")
print()
print(v96s.labeled_pos_count.sum())
print(v96a.labeled_pos_count.sum())
print(v96f.labeled_pos_count.sum())
print("done 96 s a f")
print()
print(v94s.labeled_pos_count.sum())
print(v94a.labeled_pos_count.sum())
print(v94f.labeled_pos_count.sum())
print("done 94 s a f")
print()
print(v92s.labeled_pos_count.sum())
print(v92a.labeled_pos_count.sum())
print(v92f.labeled_pos_count.sum())
print("done 92 s a f")
print()
print(v85s.labeled_pos_count.sum())
print(v85f.labeled_pos_count.sum())
print(v85a.labeled_pos_count.sum())
print("done 85 s  f a")
print()


34640
41466
15420
done 97 s a f

34640
41952
15525
done 96 s a f

34640
41714
15444
done 94 s a f

34640
40845
15238
done 92 s a f

34640
16133
40480
done 85 s  f a



In [8]:
print(v97s.missed_count_C.sum())
print(v97a.missed_count_C.sum())
print(v97f.missed_count_C.sum())
print("done 97 s a f")
print()
print(v96s.missed_count_C.sum())
print(v96a.missed_count_C.sum())
print(v96f.missed_count_C.sum())
print("done 96 s a f")
print()
print(v94s.missed_count_C.sum())
print(v94a.missed_count_C.sum())
print(v94f.missed_count_C.sum())
print("done 94 s a f")
print()
print(v92s.missed_count_C.sum())
print(v92a.missed_count_C.sum())
print(v92f.missed_count_C.sum())
print("done 92 s a f")
print()
print(v85s.missed_count_C.sum())
print(v85f.missed_count_C.sum())
print(v85a.missed_count_C.sum())
print("done 85 s  f a")
print()

3895
5259
4644
done 97 s a f

3887
5280
4665
done 96 s a f

3872
5227
4624
done 94 s a f

3877
5161
4589
done 92 s a f

3867
4852
5000
done 85 s  f a



In [9]:
print(v97s.missed_count_K.sum())
print(v97a.missed_count_K.sum())
print(v97f.missed_count_K.sum())
print("done 97 s a f")
print()
print(v96s.missed_count_K.sum())
print(v96a.missed_count_K.sum())
print(v96f.missed_count_K.sum())
print("done 96 s a f")
print()
print(v94s.missed_count_K.sum())
print(v94a.missed_count_K.sum())
print(v94f.missed_count_K.sum())
print("done 94 s a f")
print()
print(v92s.missed_count_K.sum())
print(v92a.missed_count_K.sum())
print(v92f.missed_count_K.sum())
print("done 92 s a f")
print()
print(v85s.missed_count_K.sum())
print(v85f.missed_count_K.sum())
print(v85a.missed_count_K.sum())
print("done 85 s  f a")
print()


4781
6497
5652
done 97 s a f

4769
6511
5663
done 96 s a f

4749
6481
5643
done 94 s a f

4759
6403
5583
done 92 s a f

4713
5905
6069
done 85 s  f a



In [10]:
print(v85a.missed_count.sum())
print(v85f.missed_count.sum())
print(v85s.missed_count.sum())
print("done 85 all false shared")
print()

print(v92a.missed_count.sum())
print(v92f.missed_count.sum())
print(v92s.missed_count.sum())
print("done 92 all false shared")
print()

print(v94a.missed_count.sum())
print(v94f.missed_count.sum())
print(v94s.missed_count.sum())
print("done 94 all false shared")
print()

print(v96a.missed_count.sum())
print(v96f.missed_count.sum())
print(v96s.missed_count.sum())
print("done 96 all false shared")
print()

print(v97a.missed_count.sum())
print(v97f.missed_count.sum())
print(v97s.missed_count.sum())
print("done 97 all false shared")
print()

11069
10757
8580
done 85 all false shared

11564
10172
8636
done 92 all false shared

11708
10267
8621
done 94 all false shared

11791
10328
8656
done 96 all false shared

11756
10296
8676
done 97 all false shared



In [11]:
# creating new column for each ENSP ID to have frac socre

v85a['missed_frac_C'] = v85a['missed_count_C']/v85a['count_C_targets']
v85f['missed_frac_C'] = v85f['missed_count_C']/v85f['count_C_targets']
v85s['missed_frac_C'] = v85s['missed_count_C']/v85s['count_C_targets']
print("done 85 all false shared")


v92a['missed_frac_C'] = v92a['missed_count_C']/v92a['count_C_targets']
v92f['missed_frac_C'] = v92f['missed_count_C']/v92f['count_C_targets']
v92s['missed_frac_C'] = v92s['missed_count_C']/v92s['count_C_targets']
print("done 92 all false shared")


v94a['missed_frac_C'] = v94a['missed_count_C']/v94a['count_C_targets']
v94f['missed_frac_C'] = v94f['missed_count_C']/v94f['count_C_targets']
v94s['missed_frac_C'] = v94s['missed_count_C']/v94s['count_C_targets']
print("done 94 all false shared")


v96a['missed_frac_C'] = v96a['missed_count_C']/v96a['count_C_targets']
v96f['missed_frac_C'] = v96f['missed_count_C']/v96f['count_C_targets']
v96s['missed_frac_C'] = v96s['missed_count_C']/v96s['count_C_targets']
print("done 96 all false shared")


v97a['missed_frac_C'] = v97a['missed_count_C']/v97a['count_C_targets']
v97f['missed_frac_C'] = v97f['missed_count_C']/v97f['count_C_targets']
v97s['missed_frac_C'] = v97s['missed_count_C']/v97s['count_C_targets']
print("done 97 all false shared")


done 85 all false shared
done 92 all false shared
done 94 all false shared
done 96 all false shared
done 97 all false shared


In [12]:


v85a['missed_frac_K'] = v85a['missed_count_K']/v85a['count_K_targets']
v85f['missed_frac_K'] = v85f['missed_count_K']/v85f['count_K_targets']
v85s['missed_frac_K'] = v85s['missed_count_K']/v85s['count_K_targets']
print("done 85 all false shared")


v92a['missed_frac_K'] = v92a['missed_count_K']/v92a['count_K_targets']
v92f['missed_frac_K'] = v92f['missed_count_K']/v92f['count_K_targets']
v92s['missed_frac_K'] = v92s['missed_count_K']/v92s['count_K_targets']
print("done 92 all false shared")


v94a['missed_frac_K'] = v94a['missed_count_K']/v94a['count_K_targets']
v94f['missed_frac_K'] = v94f['missed_count_K']/v94f['count_K_targets']
v94s['missed_frac_K'] = v94s['missed_count_K']/v94s['count_K_targets']
print("done 94 all false shared")


v96a['missed_frac_K'] = v96a['missed_count_K']/v96a['count_K_targets']
v96f['missed_frac_K'] = v96f['missed_count_K']/v96f['count_K_targets']
v96s['missed_frac_K'] = v96s['missed_count_K']/v96s['count_K_targets']
print("done 96 all false shared")


v97a['missed_frac_K'] = v97a['missed_count_K']/v97a['count_K_targets']
v97f['missed_frac_K'] = v97f['missed_count_K']/v97f['count_K_targets']
v97s['missed_frac_K'] = v97s['missed_count_K']/v97s['count_K_targets']
print("done 97 all false shared")


done 85 all false shared
done 92 all false shared
done 94 all false shared
done 96 all false shared
done 97 all false shared


# created ENSP specific fraction CK scores and dropping all NAN values

In [13]:
v85a_K_frac = v85a[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 
'ENSG', 'ID', 'missed_frac_K']].copy()
v85a_K_frac.dropna(inplace=True)
v85f_K_frac = v85f[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_K']].copy()
v85f_K_frac.dropna(inplace=True)
v85s_K_frac = v85s[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_K']].copy()
v85s_K_frac.dropna(inplace=True)
print("done 85 all false shared")


v92a_K_frac = v92a[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_K']].copy()
v92a_K_frac.dropna(inplace=True)
v92f_K_frac = v92f[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_K']].copy()
v92f_K_frac.dropna(inplace=True)
v92s_K_frac = v92s[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_K']].copy()
v92s_K_frac.dropna(inplace=True)
print("done 92 all false shared")


v94a_K_frac = v94a[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_K']].copy()
v94a_K_frac.dropna(inplace=True)
v94f_K_frac = v94f[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_K']].copy()
v94f_K_frac.dropna(inplace=True)
v94s_K_frac = v94s[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_K']].copy()
v94s_K_frac.dropna(inplace=True)
print("done 94 all false shared")


v96a_K_frac = v96a[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_K']].copy()
v96a_K_frac.dropna(inplace=True)
v96f_K_frac = v96f[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_K']].copy()
v96f_K_frac.dropna(inplace=True)
v96s_K_frac = v96s[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_K']].copy()
v96s_K_frac.dropna(inplace=True)
print("done 96 all false shared")


v97a_K_frac = v97a[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_K']].copy()
v97a_K_frac.dropna(inplace=True)
v97f_K_frac = v97f[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_K']].copy()
v97f_K_frac.dropna(inplace=True)
v97s_K_frac = v97s[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_K']].copy()
v97s_K_frac.dropna(inplace=True)
print("done 97 all false shared")


done 85 all false shared
done 92 all false shared
done 94 all false shared
done 96 all false shared
done 97 all false shared


In [14]:
print(v85a_K_frac.shape)
print(v85f_K_frac.shape)
print(v85s_K_frac.shape)
print("done 85 all false shared")


print(v92a_K_frac.shape)
print(v92f_K_frac.shape)
print(v92s_K_frac.shape)
print("done 92 all false shared")


print(v94a_K_frac.shape)
print(v94f_K_frac.shape)
print(v94s_K_frac.shape)
print("done 94 all false shared")


print(v96a_K_frac.shape)
print(v96f_K_frac.shape)
print(v96s_K_frac.shape)
print("done 96 all false shared")


print(v97a_K_frac.shape)
print(v97f_K_frac.shape)
print(v97s_K_frac.shape)
print("done 97 all false shared")

(6666, 8)
(2573, 8)
(5833, 8)
done 85 all false shared
(6914, 8)
(2532, 8)
(5833, 8)
done 92 all false shared
(7051, 8)
(2581, 8)
(5833, 8)
done 94 all false shared
(7086, 8)
(2596, 8)
(5833, 8)
done 96 all false shared
(7014, 8)
(2582, 8)
(5833, 8)
done 97 all false shared


In [15]:
v85a_C_frac = v85a[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 
'ENSG', 'ID', 'missed_frac_C']].copy()
v85a_C_frac.dropna(inplace=True)
v85f_C_frac = v85f[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_C']].copy()
v85f_C_frac.dropna(inplace=True)
v85s_C_frac = v85s[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_C']].copy()
v85s_C_frac.dropna(inplace=True)
print("done 85 all false shared")


v92a_C_frac = v92a[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_C']].copy()
v92a_C_frac.dropna(inplace=True)
v92f_C_frac = v92f[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_C']].copy()
v92f_C_frac.dropna(inplace=True)
v92s_C_frac = v92s[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_C']].copy()
v92s_C_frac.dropna(inplace=True)
print("done 92 all false shared")


v94a_C_frac = v94a[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_C']].copy()
v94a_C_frac.dropna(inplace=True)
v94f_C_frac = v94f[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_C']].copy()
v94f_C_frac.dropna(inplace=True)
v94s_C_frac = v94s[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_C']].copy()
v94s_C_frac.dropna(inplace=True)
print("done 94 all false shared")


v96a_C_frac = v96a[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_C']].copy()
v96a_C_frac.dropna(inplace=True)
v96f_C_frac = v96f[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_C']].copy()
v96f_C_frac.dropna(inplace=True)
v96s_C_frac = v96s[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_C']].copy()
v96s_C_frac.dropna(inplace=True)
print("done 96 all false shared")


v97a_C_frac = v97a[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_C']].copy()
v97a_C_frac.dropna(inplace=True)
v97f_C_frac = v97f[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_C']].copy()
v97f_C_frac.dropna(inplace=True)
v97s_C_frac = v97s[['ENSPv', 'ENSP', 'ENSTv', 'ENST', 'ENSGv', 'ENSG','ID', 'missed_frac_C']].copy()
v97s_C_frac.dropna(inplace=True)
print("done 97 all false shared")


done 85 all false shared
done 92 all false shared
done 94 all false shared
done 96 all false shared
done 97 all false shared


In [16]:
print(v85a_C_frac.shape)
print(v85f_C_frac.shape)
print(v85s_C_frac.shape)
print("done 85 all false shared")


print(v92a_C_frac.shape)
print(v92f_C_frac.shape)
print(v92s_C_frac.shape)
print("done 92 all false shared")


print(v94a_C_frac.shape)
print(v94f_C_frac.shape)
print(v94s_C_frac.shape)
print("done 94 all false shared")


print(v96a_C_frac.shape)
print(v96f_C_frac.shape)
print(v96s_C_frac.shape)
print("done 96 all false shared")


print(v97a_C_frac.shape)
print(v97f_C_frac.shape)
print(v97s_C_frac.shape)
print("done 97 all false shared")

(7540, 8)
(3201, 8)
(6547, 8)
done 85 all false shared
(7677, 8)
(3113, 8)
(6547, 8)
done 92 all false shared
(7847, 8)
(3157, 8)
(6547, 8)
done 94 all false shared
(7877, 8)
(3175, 8)
(6547, 8)
done 96 all false shared
(7802, 8)
(3159, 8)
(6547, 8)
done 97 all false shared


In [17]:
# saving fraction missed for letter files after dropped nan values

v85a_K_frac.to_csv("v85a_K_frac_droppedNaN.csv", index=False)
v85f_K_frac.to_csv("v85f_K_frac_droppedNaN.csv", index=False)
v85s_K_frac.to_csv("v85s_K_frac_droppedNaN.csv", index=False)


v92a_K_frac.to_csv("v92a_K_frac_droppedNaN.csv", index=False)
v92f_K_frac.to_csv("v92f_K_frac_droppedNaN.csv", index=False)
v92s_K_frac.to_csv("v92s_K_frac_droppedNaN.csv", index=False)


v94a_K_frac.to_csv("v94a_K_frac_droppedNaN.csv", index=False)
v94f_K_frac.to_csv("v94f_K_frac_droppedNaN.csv", index=False)
v94s_K_frac.to_csv("v94s_K_frac_droppedNaN.csv", index=False)


v96a_K_frac.to_csv("v96a_K_frac_droppedNaN.csv", index=False)
v96f_K_frac.to_csv("v96f_K_frac_droppedNaN.csv", index=False)
v96s_K_frac.to_csv("v96s_K_frac_droppedNaN.csv", index=False)


v97a_K_frac.to_csv("v97a_K_frac_droppedNaN.csv", index=False)
v97f_K_frac.to_csv("v97f_K_frac_droppedNaN.csv", index=False)
v97s_K_frac.to_csv("v97s_K_frac_droppedNaN.csv", index=False)

In [18]:
v85a_C_frac.to_csv("v85a_C_frac_droppedNaN.csv", index=False)
v85f_C_frac.to_csv("v85f_C_frac_droppedNaN.csv", index=False)
v85s_C_frac.to_csv("v85s_C_frac_droppedNaN.csv", index=False)


v92a_C_frac.to_csv("v92a_C_frac_droppedNaN.csv", index=False)
v92f_C_frac.to_csv("v92f_C_frac_droppedNaN.csv", index=False)
v92s_C_frac.to_csv("v92s_C_frac_droppedNaN.csv", index=False)


v94a_C_frac.to_csv("v94a_C_frac_droppedNaN.csv", index=False)
v94f_C_frac.to_csv("v94f_C_frac_droppedNaN.csv", index=False)
v94s_C_frac.to_csv("v94s_C_frac_droppedNaN.csv", index=False)


v96a_C_frac.to_csv("v96a_C_frac_droppedNaN.csv", index=False)
v96f_C_frac.to_csv("v96f_C_frac_droppedNaN.csv", index=False)
v96s_C_frac.to_csv("v96s_C_frac_droppedNaN.csv", index=False)


v97a_C_frac.to_csv("v97a_C_frac_droppedNaN.csv", index=False)
v97f_C_frac.to_csv("v97f_C_frac_droppedNaN.csv", index=False)
v97s_C_frac.to_csv("v97s_C_frac_droppedNaN.csv", index=False)

In [19]:
def uniqueCount(df, colname):
    print("len of col: ", len(df[colname]))
    print("len of col set: ", len(df[colname].unique()))
    print()

In [20]:
dfs = [v85a_C_frac, v85f_C_frac, v92a_C_frac, v92f_C_frac, v94a_C_frac, v94f_C_frac, v96a_C_frac, v96f_C_frac, v97a_C_frac, v97f_C_frac]
for i in dfs: 
    uniqueCount(i, 'ID')

len of col:  7540
len of col set:  2909

len of col:  3201
len of col set:  1365

len of col:  7677
len of col set:  2909

len of col:  3113
len of col set:  1365

len of col:  7847
len of col set:  2909

len of col:  3157
len of col set:  1365

len of col:  7877
len of col set:  2909

len of col:  3175
len of col set:  1365

len of col:  7802
len of col set:  2909

len of col:  3159
len of col set:  1365



In [22]:
dfs = [v85a_K_frac, v85f_K_frac, v92a_K_frac, v92f_K_frac, v94a_K_frac, v94f_K_frac, v96a_K_frac, v96f_K_frac, v97a_K_frac, v97f_K_frac]
for i in dfs: 
    uniqueCount(i, 'ID')

len of col:  6666
len of col set:  2618

len of col:  2573
len of col set:  1124

len of col:  6914
len of col set:  2618

len of col:  2532
len of col set:  1124

len of col:  7051
len of col set:  2618

len of col:  2581
len of col set:  1124

len of col:  7086
len of col set:  2618

len of col:  2596
len of col set:  1124

len of col:  7014
len of col set:  2618

len of col:  2582
len of col set:  1124

